# Ordering by metrics and retraining phase

## Dataset: Intel

## Experiment configuration 1


## Configuration 1
	1. Incremental guided retraining starting from scratch using the new adversarial inputs and original training set

In [ ]:
pip install --user tensorflow==2.5

In [1]:
import argparse

import numpy as np
import tensorflow as tf
import keras.backend as K

import matplotlib.pyplot as plt

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.regularizers import l2

import glob
import os
import cv2
import keras


import time
import argparse

from tqdm import tqdm

from keras.models import load_model, Model


In [2]:
tf.__version__ == '2.5.0' # Version of tensorflow

True

In [3]:
cd C:/Users/fjdur/Documents/utils/

C:\Users\fjdur\Documents\utils


In [4]:
# utils for project
import utils_guided_retraining as utils

2.5.0
keras
2.5.0


In [5]:
#cd '../notebooks/'

In [6]:
dataset = "intel"

In [7]:
x_train,y_train = utils.get_data(dataset,"Train")
x_val,y_val = utils.get_data(dataset,"Val")
x_test,y_test = utils.get_data(dataset,"Test")

In [8]:
x_train_and_adversary,y_train_and_adversary = utils.get_data(dataset,"Train_and_adversary")

In [9]:
x_adversary_training = x_train_and_adversary[len(x_train):]
print(len(x_adversary_training))
y_adversary_training = y_train_and_adversary[len(y_train):]

print(len(y_adversary_training))

3000
3000


In [10]:
# Obtaining adversarial examples for testing 
x_test_and_adversary,y_test_and_adversary = utils.get_adversarial_data(dataset,'Test_adversarial') 

In [11]:
x_adversary_test = x_test_and_adversary[len(x_test):]
print(len(x_adversary_test))
y_adversary_test = y_test_and_adversary[len(y_test):]

print(len(y_adversary_test))

3000
3000


## ----

In [12]:
# Original model 

model_dir = "C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06/"
if(dataset == 'gtsrb'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06"
elif(dataset == 'intel'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/intel_model_21_10"
elif(dataset == 'mnist'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/models2"
elif(dataset == 'cifar'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/model_02"


In [13]:
print(model_dir)
model_original = utils.My_model(dataset,True, model_dir)

C:/Users/fjdur/Documents/upc-july/models/intel_model_21_10
Model loaded correctly


In [14]:
print(x_train_and_adversary.shape)
len(x_train_and_adversary)//20

(14224, 48, 48, 1)


711

In [15]:
n_data_points = 700

## Loading LSA and DSA values 

In [16]:
lsa_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/lsa_values.npy"

In [17]:
lsa_values = np.load(lsa_direction)


In [18]:

# Obtaining top n images by LSA values
top_images_by_lsa = utils.get_x_of_indexes(list(np.flip(np.argsort(lsa_values))),x_train_and_adversary)
top_labels_by_lsa = utils.get_x_of_indexes(list(np.flip(np.argsort(lsa_values))),y_train_and_adversary)


In [19]:
m = n_data_points
n = 0
image_sets_lsa = []
label_sets_lsa = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_lsa)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_lsa)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_lsa)%m))
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m])
    image_sets_lsa.append(top_images_by_lsa_n)
    label_sets_lsa.append(top_labels_by_lsa_n)
    print(len(top_images_by_lsa_n))
    n += m

0 :
0  ->  700
700
1 :
0  ->  1400
1400
2 :
0  ->  2100
2100
3 :
0  ->  2800
2800
4 :
0  ->  3500
3500
5 :
0  ->  4200
4200
6 :
0  ->  4900
4900
7 :
0  ->  5600
5600
8 :
0  ->  6300
6300
9 :
0  ->  7000
7000
10 :
0  ->  7700
7700
11 :
0  ->  8400
8400
12 :
0  ->  9100
9100
13 :
0  ->  9800
9800
14 :
0  ->  10500
10500
15 :
0  ->  11200
11200
16 :
0  ->  11900
11900
17 :
0  ->  12600
12600
18 :
0  ->  13300
13300
19 :
0  ->  14000
14000
20 :
Last
0  ->  14924
14224


In [20]:
len(image_sets_lsa)

21

## Training guided by LSA

In [21]:
len(label_sets_lsa)

21

In [22]:
print(model_dir)

models_lsa = []
for i in range(len(label_sets_lsa)):
    print(i,":")
    model = utils.My_model(dataset,False,None)
    model.compile_model()
    models_lsa.append(model)


C:/Users/fjdur/Documents/upc-july/models/intel_model_21_10
0 :
Model compiled
1 :
Model compiled
2 :
Model compiled
3 :
Model compiled
4 :
Model compiled
5 :
Model compiled
6 :
Model compiled
7 :
Model compiled
8 :
Model compiled
9 :
Model compiled
10 :
Model compiled
11 :
Model compiled
12 :
Model compiled
13 :
Model compiled
14 :
Model compiled
15 :
Model compiled
16 :
Model compiled
17 :
Model compiled
18 :
Model compiled
19 :
Model compiled
20 :
Model compiled


In [23]:
n = 0
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n=n+1

Epoch 1/20
6/6 [==============================] - 67s 1s/step - loss: 1.7803 - accuracy: 0.2201 - val_loss: 1.9926 - val_accuracy: 0.1562
Epoch 2/20
6/6 [==============================] - 4s 813ms/step - loss: 1.7546 - accuracy: 0.2746 - val_loss: 1.9462 - val_accuracy: 0.1562
Epoch 3/20
6/6 [==============================] - 4s 835ms/step - loss: 1.7210 - accuracy: 0.2826 - val_loss: 1.8062 - val_accuracy: 0.1562
Epoch 4/20
6/6 [==============================] - 5s 830ms/step - loss: 1.7235 - accuracy: 0.2704 - val_loss: 1.9015 - val_accuracy: 0.1562
Epoch 5/20
6/6 [==============================] - 5s 834ms/step - loss: 1.6603 - accuracy: 0.2720 - val_loss: 1.7668 - val_accuracy: 0.1566
Epoch 6/20
6/6 [==============================] - 5s 830ms/step - loss: 1.6456 - accuracy: 0.3059 - val_loss: 1.7747 - val_accuracy: 0.1562
Epoch 7/20
6/6 [==============================] - 5s 832ms/step - loss: 1.6551 - accuracy: 0.3010 - val_loss: 1.7594 - val_accuracy: 0.1651
Epoch 8/20
6/6 [======

In [24]:
n=1
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

Epoch 1/20
11/11 [==============================] - 8s 438ms/step - loss: 1.7824 - accuracy: 0.4001 - val_loss: 2.1771 - val_accuracy: 0.1562
Epoch 2/20
11/11 [==============================] - 4s 423ms/step - loss: 1.3819 - accuracy: 0.5643 - val_loss: 2.0205 - val_accuracy: 0.1562
Epoch 3/20
11/11 [==============================] - 5s 453ms/step - loss: 1.3377 - accuracy: 0.5679 - val_loss: 1.9553 - val_accuracy: 0.1562
Epoch 4/20
11/11 [==============================] - 5s 470ms/step - loss: 1.3806 - accuracy: 0.5546 - val_loss: 2.0490 - val_accuracy: 0.1562
Epoch 5/20
11/11 [==============================] - 5s 467ms/step - loss: 1.3675 - accuracy: 0.5502 - val_loss: 1.9605 - val_accuracy: 0.1562
Epoch 6/20
11/11 [==============================] - 5s 445ms/step - loss: 1.4016 - accuracy: 0.5577 - val_loss: 2.0491 - val_accuracy: 0.1562
Epoch 7/20
11/11 [==============================] - 5s 445ms/step - loss: 1.3843 - accuracy: 0.5406 - val_loss: 1.9689 - val_accuracy: 0.1562
Epoch 

In [25]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

2
Epoch 1/20
17/17 [==============================] - 11s 403ms/step - loss: 1.9041 - accuracy: 0.5537 - val_loss: 2.1541 - val_accuracy: 0.1562
Epoch 2/20
17/17 [==============================] - 7s 400ms/step - loss: 1.1042 - accuracy: 0.6885 - val_loss: 2.8940 - val_accuracy: 0.1562
Epoch 3/20
17/17 [==============================] - 7s 394ms/step - loss: 1.1501 - accuracy: 0.6775 - val_loss: 2.2250 - val_accuracy: 0.1562
Epoch 4/20
17/17 [==============================] - 7s 389ms/step - loss: 1.1127 - accuracy: 0.6835 - val_loss: 1.9750 - val_accuracy: 0.1562
Epoch 5/20
17/17 [==============================] - 7s 392ms/step - loss: 1.1156 - accuracy: 0.6815 - val_loss: 1.8824 - val_accuracy: 0.1562
Epoch 6/20
17/17 [==============================] - 7s 399ms/step - loss: 1.1611 - accuracy: 0.6623 - val_loss: 2.1463 - val_accuracy: 0.1562
Epoch 7/20
17/17 [==============================] - 7s 393ms/step - loss: 1.0407 - accuracy: 0.6767 - val_loss: 2.9065 - val_accuracy: 0.1562
Epo

In [26]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

3
Epoch 1/20
22/22 [==============================] - 13s 401ms/step - loss: 1.4597 - accuracy: 0.6339 - val_loss: 1.9498 - val_accuracy: 0.1562
Epoch 2/20
22/22 [==============================] - 8s 382ms/step - loss: 1.1048 - accuracy: 0.7156 - val_loss: 1.9618 - val_accuracy: 0.1562
Epoch 3/20
22/22 [==============================] - 8s 368ms/step - loss: 1.0297 - accuracy: 0.7305 - val_loss: 2.1837 - val_accuracy: 0.1562
Epoch 4/20
22/22 [==============================] - 8s 360ms/step - loss: 1.0238 - accuracy: 0.7189 - val_loss: 2.4882 - val_accuracy: 0.1562
Epoch 5/20
22/22 [==============================] - 8s 388ms/step - loss: 1.0328 - accuracy: 0.7268 - val_loss: 2.3796 - val_accuracy: 0.1562
Epoch 6/20
22/22 [==============================] - 8s 360ms/step - loss: 1.0371 - accuracy: 0.7056 - val_loss: 2.1998 - val_accuracy: 0.1562
Epoch 7/20
22/22 [==============================] - 8s 358ms/step - loss: 1.0489 - accuracy: 0.7006 - val_loss: 1.9036 - val_accuracy: 0.1562
Epo

In [27]:
print(n) #4
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

4
Epoch 1/20
28/28 [==============================] - 12s 330ms/step - loss: 1.4520 - accuracy: 0.4902 - val_loss: 2.2643 - val_accuracy: 0.1562
Epoch 2/20
28/28 [==============================] - 9s 334ms/step - loss: 1.2904 - accuracy: 0.5777 - val_loss: 2.0587 - val_accuracy: 0.1562
Epoch 3/20
28/28 [==============================] - 9s 334ms/step - loss: 1.2181 - accuracy: 0.5877 - val_loss: 1.9585 - val_accuracy: 0.1705
Epoch 4/20
28/28 [==============================] - 9s 334ms/step - loss: 1.1707 - accuracy: 0.5971 - val_loss: 2.1065 - val_accuracy: 0.2322
Epoch 5/20
28/28 [==============================] - 9s 337ms/step - loss: 1.1362 - accuracy: 0.6036 - val_loss: 1.7181 - val_accuracy: 0.2411
Epoch 6/20
28/28 [==============================] - 9s 335ms/step - loss: 1.0229 - accuracy: 0.6362 - val_loss: 1.5388 - val_accuracy: 0.3088
Epoch 7/20
28/28 [==============================] - 9s 338ms/step - loss: 0.9714 - accuracy: 0.6522 - val_loss: 1.6451 - val_accuracy: 0.3099
Epo

In [28]:
n=5
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

5
Epoch 1/20
33/33 [==============================] - 14s 323ms/step - loss: 1.4612 - accuracy: 0.4171 - val_loss: 2.4496 - val_accuracy: 0.1687
Epoch 2/20
33/33 [==============================] - 11s 330ms/step - loss: 1.2411 - accuracy: 0.5141 - val_loss: 2.2715 - val_accuracy: 0.2297
Epoch 3/20
33/33 [==============================] - 11s 330ms/step - loss: 1.1308 - accuracy: 0.5804 - val_loss: 1.7522 - val_accuracy: 0.2129
Epoch 4/20
33/33 [==============================] - 11s 330ms/step - loss: 1.0368 - accuracy: 0.6141 - val_loss: 1.5447 - val_accuracy: 0.2593
Epoch 5/20
33/33 [==============================] - 11s 333ms/step - loss: 0.9916 - accuracy: 0.6285 - val_loss: 1.6600 - val_accuracy: 0.2685
Epoch 6/20
33/33 [==============================] - 11s 332ms/step - loss: 0.9330 - accuracy: 0.6378 - val_loss: 1.3418 - val_accuracy: 0.3959
Epoch 7/20
33/33 [==============================] - 11s 331ms/step - loss: 0.9162 - accuracy: 0.6667 - val_loss: 1.2979 - val_accuracy: 0.42

In [29]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

6
Epoch 1/20
39/39 [==============================] - 15s 318ms/step - loss: 1.4821 - accuracy: 0.4131 - val_loss: 2.2749 - val_accuracy: 0.1719
Epoch 2/20
39/39 [==============================] - 13s 324ms/step - loss: 1.2095 - accuracy: 0.4991 - val_loss: 3.3761 - val_accuracy: 0.2400
Epoch 3/20
39/39 [==============================] - 12s 319ms/step - loss: 1.1146 - accuracy: 0.6183 - val_loss: 1.7308 - val_accuracy: 0.2457
Epoch 4/20
39/39 [==============================] - 12s 320ms/step - loss: 0.9773 - accuracy: 0.6450 - val_loss: 1.6852 - val_accuracy: 0.2589
Epoch 5/20
39/39 [==============================] - 12s 320ms/step - loss: 0.9013 - accuracy: 0.6653 - val_loss: 1.5217 - val_accuracy: 0.2757
Epoch 6/20
39/39 [==============================] - 12s 321ms/step - loss: 0.8436 - accuracy: 0.6832 - val_loss: 1.6476 - val_accuracy: 0.2561
Epoch 7/20
39/39 [==============================] - 12s 321ms/step - loss: 0.7855 - accuracy: 0.7031 - val_loss: 1.4338 - val_accuracy: 0.37

In [30]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

7
Epoch 1/20
44/44 [==============================] - 17s 319ms/step - loss: 1.5515 - accuracy: 0.3706 - val_loss: 2.0086 - val_accuracy: 0.1723
Epoch 2/20
44/44 [==============================] - 14s 316ms/step - loss: 1.3057 - accuracy: 0.4799 - val_loss: 2.2549 - val_accuracy: 0.2011
Epoch 3/20
44/44 [==============================] - 14s 317ms/step - loss: 1.1539 - accuracy: 0.5666 - val_loss: 1.6305 - val_accuracy: 0.2971
Epoch 4/20
44/44 [==============================] - 14s 317ms/step - loss: 1.0306 - accuracy: 0.6155 - val_loss: 1.5362 - val_accuracy: 0.3442
Epoch 5/20
44/44 [==============================] - 14s 320ms/step - loss: 0.9251 - accuracy: 0.6493 - val_loss: 1.3769 - val_accuracy: 0.3880
Epoch 6/20
44/44 [==============================] - 14s 321ms/step - loss: 0.9091 - accuracy: 0.6486 - val_loss: 1.3072 - val_accuracy: 0.4479
Epoch 7/20
44/44 [==============================] - 14s 317ms/step - loss: 0.8498 - accuracy: 0.6857 - val_loss: 1.5316 - val_accuracy: 0.33

In [31]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

8
Epoch 1/20
50/50 [==============================] - 18s 306ms/step - loss: 1.5393 - accuracy: 0.3488 - val_loss: 1.9422 - val_accuracy: 0.1858
Epoch 2/20
50/50 [==============================] - 15s 311ms/step - loss: 1.3063 - accuracy: 0.4919 - val_loss: 1.5164 - val_accuracy: 0.3156
Epoch 3/20
50/50 [==============================] - 15s 309ms/step - loss: 1.1013 - accuracy: 0.6008 - val_loss: 1.4529 - val_accuracy: 0.3588
Epoch 4/20
50/50 [==============================] - 16s 311ms/step - loss: 1.0270 - accuracy: 0.6245 - val_loss: 1.5391 - val_accuracy: 0.3138
Epoch 5/20
50/50 [==============================] - 16s 315ms/step - loss: 0.9864 - accuracy: 0.6341 - val_loss: 1.3992 - val_accuracy: 0.4051
Epoch 6/20
50/50 [==============================] - 16s 327ms/step - loss: 0.9395 - accuracy: 0.6480 - val_loss: 1.2717 - val_accuracy: 0.4076
Epoch 7/20
50/50 [==============================] - 17s 335ms/step - loss: 0.8881 - accuracy: 0.6786 - val_loss: 1.2287 - val_accuracy: 0.45

In [32]:
print(n)#9
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

9
Epoch 1/20
55/55 [==============================] - 22s 332ms/step - loss: 1.5974 - accuracy: 0.3029 - val_loss: 1.7982 - val_accuracy: 0.2842
Epoch 2/20
55/55 [==============================] - 18s 334ms/step - loss: 1.2610 - accuracy: 0.5259 - val_loss: 1.6480 - val_accuracy: 0.3320
Epoch 3/20
55/55 [==============================] - 18s 334ms/step - loss: 1.1479 - accuracy: 0.5830 - val_loss: 1.4775 - val_accuracy: 0.3598
Epoch 4/20
55/55 [==============================] - 18s 333ms/step - loss: 1.0245 - accuracy: 0.6252 - val_loss: 1.4033 - val_accuracy: 0.4034
Epoch 5/20
55/55 [==============================] - 18s 334ms/step - loss: 0.9664 - accuracy: 0.6514 - val_loss: 1.3471 - val_accuracy: 0.3994
Epoch 6/20
55/55 [==============================] - 19s 338ms/step - loss: 0.9318 - accuracy: 0.6532 - val_loss: 1.3074 - val_accuracy: 0.4066
Epoch 7/20
55/55 [==============================] - 18s 333ms/step - loss: 0.8845 - accuracy: 0.6735 - val_loss: 1.4554 - val_accuracy: 0.41

In [33]:
## print(n)
n=10
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

Epoch 1/20
61/61 [==============================] - 24s 326ms/step - loss: 1.5942 - accuracy: 0.3108 - val_loss: 2.0465 - val_accuracy: 0.3295
Epoch 2/20
61/61 [==============================] - 20s 330ms/step - loss: 1.2824 - accuracy: 0.5291 - val_loss: 1.5427 - val_accuracy: 0.3709
Epoch 3/20
61/61 [==============================] - 20s 329ms/step - loss: 1.0883 - accuracy: 0.6073 - val_loss: 1.4400 - val_accuracy: 0.4444
Epoch 4/20
61/61 [==============================] - 20s 327ms/step - loss: 1.0308 - accuracy: 0.6346 - val_loss: 1.7817 - val_accuracy: 0.4001
Epoch 5/20
61/61 [==============================] - 20s 331ms/step - loss: 0.9887 - accuracy: 0.6536 - val_loss: 1.3882 - val_accuracy: 0.4094
Epoch 6/20
61/61 [==============================] - 20s 330ms/step - loss: 0.9286 - accuracy: 0.6626 - val_loss: 1.8671 - val_accuracy: 0.3752
Epoch 7/20
61/61 [==============================] - 20s 329ms/step - loss: 0.8591 - accuracy: 0.6951 - val_loss: 1.4979 - val_accuracy: 0.3698

In [34]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

11
Epoch 1/20
66/66 [==============================] - 26s 328ms/step - loss: 1.6031 - accuracy: 0.2943 - val_loss: 2.0638 - val_accuracy: 0.2129
Epoch 2/20
66/66 [==============================] - 22s 328ms/step - loss: 1.2873 - accuracy: 0.5172 - val_loss: 1.6242 - val_accuracy: 0.3976
Epoch 3/20
66/66 [==============================] - 22s 332ms/step - loss: 1.1251 - accuracy: 0.5808 - val_loss: 1.4181 - val_accuracy: 0.4330
Epoch 4/20
66/66 [==============================] - 22s 327ms/step - loss: 1.0194 - accuracy: 0.6148 - val_loss: 1.2307 - val_accuracy: 0.4576
Epoch 5/20
66/66 [==============================] - 22s 327ms/step - loss: 0.9728 - accuracy: 0.6409 - val_loss: 1.3030 - val_accuracy: 0.4782
Epoch 6/20
66/66 [==============================] - 22s 330ms/step - loss: 0.9031 - accuracy: 0.6675 - val_loss: 1.2923 - val_accuracy: 0.4964
Epoch 7/20
66/66 [==============================] - 22s 328ms/step - loss: 0.8553 - accuracy: 0.6932 - val_loss: 1.2677 - val_accuracy: 0.5

In [35]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

12
Epoch 1/20
72/72 [==============================] - 27s 321ms/step - loss: 1.6130 - accuracy: 0.2807 - val_loss: 1.7772 - val_accuracy: 0.3178
Epoch 2/20
72/72 [==============================] - 23s 323ms/step - loss: 1.2544 - accuracy: 0.5345 - val_loss: 1.4073 - val_accuracy: 0.4215
Epoch 3/20
72/72 [==============================] - 25s 346ms/step - loss: 1.0948 - accuracy: 0.5939 - val_loss: 1.3871 - val_accuracy: 0.4508
Epoch 4/20
72/72 [==============================] - 24s 331ms/step - loss: 1.0141 - accuracy: 0.6229 - val_loss: 1.4706 - val_accuracy: 0.4722
Epoch 5/20
72/72 [==============================] - 23s 324ms/step - loss: 0.9642 - accuracy: 0.6457 - val_loss: 1.2759 - val_accuracy: 0.4936
Epoch 6/20
72/72 [==============================] - 23s 324ms/step - loss: 0.9061 - accuracy: 0.6712 - val_loss: 1.5658 - val_accuracy: 0.4358
Epoch 7/20
72/72 [==============================] - 23s 325ms/step - loss: 0.8456 - accuracy: 0.6973 - val_loss: 1.0747 - val_accuracy: 0.5

In [36]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

13
Epoch 1/20
77/77 [==============================] - 29s 323ms/step - loss: 1.6047 - accuracy: 0.3113 - val_loss: 1.5695 - val_accuracy: 0.3934
Epoch 2/20
77/77 [==============================] - 25s 324ms/step - loss: 1.2150 - accuracy: 0.5562 - val_loss: 1.3900 - val_accuracy: 0.4586
Epoch 3/20
77/77 [==============================] - 25s 324ms/step - loss: 1.0594 - accuracy: 0.6043 - val_loss: 1.2410 - val_accuracy: 0.5143
Epoch 4/20
77/77 [==============================] - 25s 325ms/step - loss: 0.9760 - accuracy: 0.6408 - val_loss: 1.3183 - val_accuracy: 0.5053
Epoch 5/20
77/77 [==============================] - 25s 325ms/step - loss: 0.9117 - accuracy: 0.6698 - val_loss: 1.3393 - val_accuracy: 0.5350
Epoch 6/20
77/77 [==============================] - 25s 324ms/step - loss: 0.8512 - accuracy: 0.6948 - val_loss: 1.2226 - val_accuracy: 0.5218
Epoch 7/20
77/77 [==============================] - 25s 327ms/step - loss: 0.8320 - accuracy: 0.7003 - val_loss: 1.1379 - val_accuracy: 0.5

In [37]:
print(n)#14
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

14
Epoch 1/20
83/83 [==============================] - 32s 321ms/step - loss: 1.6723 - accuracy: 0.3096 - val_loss: 1.8125 - val_accuracy: 0.3869
Epoch 2/20
83/83 [==============================] - 26s 319ms/step - loss: 1.2980 - accuracy: 0.5108 - val_loss: 1.3840 - val_accuracy: 0.4579
Epoch 3/20
83/83 [==============================] - 27s 321ms/step - loss: 1.1346 - accuracy: 0.5742 - val_loss: 2.6714 - val_accuracy: 0.2750
Epoch 4/20
83/83 [==============================] - 27s 320ms/step - loss: 1.1287 - accuracy: 0.5887 - val_loss: 1.3403 - val_accuracy: 0.4608
Epoch 5/20
83/83 [==============================] - 26s 319ms/step - loss: 0.9778 - accuracy: 0.6341 - val_loss: 1.3779 - val_accuracy: 0.3631
Epoch 6/20
83/83 [==============================] - 27s 320ms/step - loss: 0.9693 - accuracy: 0.6314 - val_loss: 1.3717 - val_accuracy: 0.4811
Epoch 7/20
83/83 [==============================] - 27s 320ms/step - loss: 0.9245 - accuracy: 0.6561 - val_loss: 1.0423 - val_accuracy: 0.5

In [38]:
n=15
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

15
Epoch 1/20
88/88 [==============================] - 33s 325ms/step - loss: 1.6270 - accuracy: 0.2978 - val_loss: 1.5658 - val_accuracy: 0.3666
Epoch 2/20
88/88 [==============================] - 28s 321ms/step - loss: 1.2705 - accuracy: 0.4990 - val_loss: 1.3869 - val_accuracy: 0.4665
Epoch 3/20
88/88 [==============================] - 28s 320ms/step - loss: 1.1270 - accuracy: 0.5579 - val_loss: 1.1799 - val_accuracy: 0.5189
Epoch 4/20
88/88 [==============================] - 28s 319ms/step - loss: 1.0571 - accuracy: 0.5897 - val_loss: 1.1718 - val_accuracy: 0.5193
Epoch 5/20
88/88 [==============================] - 28s 322ms/step - loss: 0.9807 - accuracy: 0.6139 - val_loss: 1.0815 - val_accuracy: 0.5471
Epoch 6/20
88/88 [==============================] - 28s 322ms/step - loss: 0.9556 - accuracy: 0.6357 - val_loss: 1.0818 - val_accuracy: 0.5720
Epoch 7/20
88/88 [==============================] - 28s 321ms/step - loss: 0.9095 - accuracy: 0.6473 - val_loss: 0.9727 - val_accuracy: 0.6

In [39]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

16
Epoch 1/20
93/93 [==============================] - 34s 325ms/step - loss: 1.6237 - accuracy: 0.2848 - val_loss: 1.4893 - val_accuracy: 0.4354
Epoch 2/20
93/93 [==============================] - 30s 322ms/step - loss: 1.2749 - accuracy: 0.4841 - val_loss: 1.3649 - val_accuracy: 0.4330
Epoch 3/20
93/93 [==============================] - 30s 321ms/step - loss: 1.1161 - accuracy: 0.5558 - val_loss: 1.0843 - val_accuracy: 0.5410
Epoch 4/20
93/93 [==============================] - 30s 321ms/step - loss: 1.0718 - accuracy: 0.5816 - val_loss: 1.1144 - val_accuracy: 0.5613
Epoch 5/20
93/93 [==============================] - 30s 324ms/step - loss: 0.9664 - accuracy: 0.6210 - val_loss: 1.1045 - val_accuracy: 0.5275
Epoch 6/20
93/93 [==============================] - 31s 328ms/step - loss: 0.9175 - accuracy: 0.6423 - val_loss: 1.1792 - val_accuracy: 0.5617
Epoch 7/20
93/93 [==============================] - 30s 321ms/step - loss: 0.8579 - accuracy: 0.6766 - val_loss: 1.0368 - val_accuracy: 0.6

In [40]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

17
Epoch 1/20
99/99 [==============================] - 36s 317ms/step - loss: 1.6763 - accuracy: 0.2624 - val_loss: 1.2284 - val_accuracy: 0.4936
Epoch 2/20
99/99 [==============================] - 31s 318ms/step - loss: 1.2974 - accuracy: 0.4677 - val_loss: 1.5332 - val_accuracy: 0.3766
Epoch 3/20
99/99 [==============================] - 31s 318ms/step - loss: 1.1675 - accuracy: 0.5368 - val_loss: 1.0683 - val_accuracy: 0.5760
Epoch 4/20
99/99 [==============================] - 32s 322ms/step - loss: 1.0801 - accuracy: 0.5743 - val_loss: 0.8964 - val_accuracy: 0.6819
Epoch 5/20
99/99 [==============================] - 31s 317ms/step - loss: 1.0136 - accuracy: 0.6030 - val_loss: 0.8097 - val_accuracy: 0.6972
Epoch 6/20
99/99 [==============================] - 32s 318ms/step - loss: 0.9539 - accuracy: 0.6306 - val_loss: 0.8744 - val_accuracy: 0.7015
Epoch 7/20
99/99 [==============================] - 31s 317ms/step - loss: 0.9101 - accuracy: 0.6562 - val_loss: 0.9917 - val_accuracy: 0.6

In [41]:
time.sleep(400)

In [42]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

18
Epoch 1/20
104/104 [==============================] - 35s 295ms/step - loss: 1.6832 - accuracy: 0.2782 - val_loss: 1.2074 - val_accuracy: 0.5353
Epoch 2/20
104/104 [==============================] - 30s 289ms/step - loss: 1.2909 - accuracy: 0.4684 - val_loss: 1.1085 - val_accuracy: 0.5371
Epoch 3/20
104/104 [==============================] - 30s 289ms/step - loss: 1.1436 - accuracy: 0.5364 - val_loss: 0.9840 - val_accuracy: 0.6049
Epoch 4/20
104/104 [==============================] - 30s 290ms/step - loss: 1.0393 - accuracy: 0.5931 - val_loss: 0.8967 - val_accuracy: 0.6630
Epoch 5/20
104/104 [==============================] - 30s 289ms/step - loss: 0.9701 - accuracy: 0.6189 - val_loss: 0.8010 - val_accuracy: 0.7136
Epoch 6/20
104/104 [==============================] - 30s 286ms/step - loss: 0.9118 - accuracy: 0.6465 - val_loss: 0.7617 - val_accuracy: 0.7229
Epoch 7/20
104/104 [==============================] - 30s 289ms/step - loss: 0.8458 - accuracy: 0.6820 - val_loss: 0.6995 - val

In [43]:
time.sleep(400)

In [44]:
print(n)#19
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

19
Epoch 1/20
110/110 [==============================] - 36s 292ms/step - loss: 1.6632 - accuracy: 0.2769 - val_loss: 1.7745 - val_accuracy: 0.3673
Epoch 2/20
110/110 [==============================] - 32s 288ms/step - loss: 1.2679 - accuracy: 0.5007 - val_loss: 1.0619 - val_accuracy: 0.5881
Epoch 3/20
110/110 [==============================] - 32s 288ms/step - loss: 1.1252 - accuracy: 0.5653 - val_loss: 0.8921 - val_accuracy: 0.6726
Epoch 4/20
110/110 [==============================] - 32s 288ms/step - loss: 1.0187 - accuracy: 0.6091 - val_loss: 0.8383 - val_accuracy: 0.6994
Epoch 5/20
110/110 [==============================] - 32s 292ms/step - loss: 0.9245 - accuracy: 0.6567 - val_loss: 0.7812 - val_accuracy: 0.7111
Epoch 6/20
110/110 [==============================] - 33s 301ms/step - loss: 0.8819 - accuracy: 0.6664 - val_loss: 0.6936 - val_accuracy: 0.7450
Epoch 7/20
110/110 [==============================] - 34s 313ms/step - loss: 0.8276 - accuracy: 0.6936 - val_loss: 0.7425 - val

In [45]:
time.sleep(400)

In [46]:
print(n)#last
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

20
Epoch 1/20
112/112 [==============================] - 37s 290ms/step - loss: 1.6743 - accuracy: 0.2724 - val_loss: 1.2214 - val_accuracy: 0.5210
Epoch 2/20
112/112 [==============================] - 32s 286ms/step - loss: 1.2600 - accuracy: 0.5028 - val_loss: 1.0411 - val_accuracy: 0.5913
Epoch 3/20
112/112 [==============================] - 32s 286ms/step - loss: 1.0772 - accuracy: 0.5710 - val_loss: 0.9695 - val_accuracy: 0.6373
Epoch 4/20
112/112 [==============================] - 32s 285ms/step - loss: 0.9930 - accuracy: 0.6185 - val_loss: 0.7843 - val_accuracy: 0.7101
Epoch 5/20
112/112 [==============================] - 32s 287ms/step - loss: 0.9208 - accuracy: 0.6498 - val_loss: 1.0295 - val_accuracy: 0.5774
Epoch 6/20
112/112 [==============================] - 32s 285ms/step - loss: 0.8671 - accuracy: 0.6747 - val_loss: 0.7860 - val_accuracy: 0.7108
Epoch 7/20
112/112 [==============================] - 32s 287ms/step - loss: 0.7920 - accuracy: 0.7087 - val_loss: 1.3733 - val

In [47]:
len(models_lsa)

21

In [48]:
#new_model_lsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_lsa_e2"
new_model_lsa_dir  = "D:/models/aug_22/"+dataset+"/C1/"+dataset+"_model_c1_aug_lsa_e1"

i=0

for model in models_lsa:
    model.save(new_model_lsa_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_lsa_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_lsa_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_lsa_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_lsa_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_lsa_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_lsa_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_lsa_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_lsa_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/int

In [49]:
import gc

In [50]:
try:
    del lsa_values
    del top_images_by_lsa
    del top_labels_by_lsa
    del image_sets_lsa
    del label_sets_lsa
    del models_lsa
except:
    print("Error")

In [51]:
gc.collect()

120965

In [ ]:
loading = False

models_lsa = []

if loading:
    for i in range(10,20):
        #model_lsa_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_lsa_e1_"+str(i)
        print(model_lsa_dir)
        model =utils.My_model(dataset,True,model_lsa_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_lsa.append(model)
       
    

## Training guided by DSA

In [52]:
dsa_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/dsa_values.npy"

In [53]:
dsa_values = np.load(dsa_direction)

In [54]:
top_images_by_dsa = utils.get_x_of_indexes(list(np.flip(np.argsort(dsa_values))),x_train_and_adversary)
top_labels_by_dsa = utils.get_x_of_indexes(list(np.flip(np.argsort(dsa_values))),y_train_and_adversary)

In [55]:
m = n_data_points
n = 0
image_sets_dsa = []
label_sets_dsa = []


for i in range((len(top_images_by_dsa)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_dsa)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_dsa)%m))
        top_images_by_dsa_n = np.array(top_images_by_dsa[:n+m+(len(top_images_by_dsa)%m)])
        top_labels_by_dsa_n = np.array(top_labels_by_dsa[:n+m+(len(top_images_by_dsa)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_dsa_n = np.array(top_images_by_dsa[:n+m])
        top_labels_by_dsa_n = np.array(top_labels_by_dsa[:n+m])
    image_sets_dsa.append(top_images_by_dsa_n)
    label_sets_dsa.append(top_labels_by_dsa_n)
    print(len(top_images_by_dsa_n))
    n += m



0 :
0  ->  700
700
1 :
0  ->  1400
1400
2 :
0  ->  2100
2100
3 :
0  ->  2800
2800
4 :
0  ->  3500
3500
5 :
0  ->  4200
4200
6 :
0  ->  4900
4900
7 :
0  ->  5600
5600
8 :
0  ->  6300
6300
9 :
0  ->  7000
7000
10 :
0  ->  7700
7700
11 :
0  ->  8400
8400
12 :
0  ->  9100
9100
13 :
0  ->  9800
9800
14 :
0  ->  10500
10500
15 :
0  ->  11200
11200
16 :
0  ->  11900
11900
17 :
0  ->  12600
12600
18 :
0  ->  13300
13300
19 :
0  ->  14000
14000
20 :
Last
0  ->  14924
14224


In [56]:
print(model_dir)

models_dsa = []
for i in range(len(label_sets_dsa)):
    print(i,":")
    model = utils.My_model(dataset,False,None)
    model.compile_model()
    models_dsa.append(model)


C:/Users/fjdur/Documents/upc-july/models/intel_model_21_10
0 :
Model compiled
1 :
Model compiled
2 :
Model compiled
3 :
Model compiled
4 :
Model compiled
5 :
Model compiled
6 :
Model compiled
7 :
Model compiled
8 :
Model compiled
9 :
Model compiled
10 :
Model compiled
11 :
Model compiled
12 :
Model compiled
13 :
Model compiled
14 :
Model compiled
15 :
Model compiled
16 :
Model compiled
17 :
Model compiled
18 :
Model compiled
19 :
Model compiled
20 :
Model compiled


In [57]:
n=0
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

0
Epoch 1/20
6/6 [==============================] - 8s 725ms/step - loss: 1.8198 - accuracy: 0.1648 - val_loss: 1.7871 - val_accuracy: 0.1623
Epoch 2/20
6/6 [==============================] - 3s 575ms/step - loss: 1.7782 - accuracy: 0.1991 - val_loss: 1.7931 - val_accuracy: 0.2126
Epoch 3/20
6/6 [==============================] - 3s 582ms/step - loss: 1.7783 - accuracy: 0.1980 - val_loss: 1.7400 - val_accuracy: 0.3142
Epoch 4/20
6/6 [==============================] - 3s 584ms/step - loss: 1.7308 - accuracy: 0.2725 - val_loss: 3.5563 - val_accuracy: 0.1673
Epoch 5/20
6/6 [==============================] - 3s 583ms/step - loss: 2.7317 - accuracy: 0.2303 - val_loss: 1.6385 - val_accuracy: 0.2835
Epoch 6/20
6/6 [==============================] - 3s 578ms/step - loss: 1.5999 - accuracy: 0.3270 - val_loss: 1.4973 - val_accuracy: 0.3588
Epoch 7/20
6/6 [==============================] - 3s 580ms/step - loss: 1.5442 - accuracy: 0.3785 - val_loss: 1.5042 - val_accuracy: 0.4069
Epoch 8/20
6/6 [==

In [58]:
n=1
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

1
Epoch 1/20
11/11 [==============================] - 9s 526ms/step - loss: 1.8025 - accuracy: 0.1805 - val_loss: 1.7774 - val_accuracy: 0.1790
Epoch 2/20
11/11 [==============================] - 5s 435ms/step - loss: 1.7600 - accuracy: 0.2317 - val_loss: 1.6947 - val_accuracy: 0.2347
Epoch 3/20
11/11 [==============================] - 5s 472ms/step - loss: 1.7258 - accuracy: 0.2671 - val_loss: 1.5663 - val_accuracy: 0.3880
Epoch 4/20
11/11 [==============================] - 5s 479ms/step - loss: 1.6330 - accuracy: 0.3141 - val_loss: 1.5578 - val_accuracy: 0.3481
Epoch 5/20
11/11 [==============================] - 5s 481ms/step - loss: 1.5805 - accuracy: 0.3349 - val_loss: 1.4883 - val_accuracy: 0.3609
Epoch 6/20
11/11 [==============================] - 5s 486ms/step - loss: 1.5203 - accuracy: 0.3459 - val_loss: 1.4602 - val_accuracy: 0.4223
Epoch 7/20
11/11 [==============================] - 5s 480ms/step - loss: 1.5208 - accuracy: 0.3594 - val_loss: 1.4319 - val_accuracy: 0.4519
Epoc

In [59]:
n=2
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

2
Epoch 1/20
17/17 [==============================] - 11s 420ms/step - loss: 1.8026 - accuracy: 0.2024 - val_loss: 1.7419 - val_accuracy: 0.2143
Epoch 2/20
17/17 [==============================] - 7s 414ms/step - loss: 1.7317 - accuracy: 0.2703 - val_loss: 1.3984 - val_accuracy: 0.4169
Epoch 3/20
17/17 [==============================] - 7s 416ms/step - loss: 1.6082 - accuracy: 0.3414 - val_loss: 1.3829 - val_accuracy: 0.4476
Epoch 4/20
17/17 [==============================] - 7s 414ms/step - loss: 1.4934 - accuracy: 0.3980 - val_loss: 1.2781 - val_accuracy: 0.4925
Epoch 5/20
17/17 [==============================] - 7s 412ms/step - loss: 1.4783 - accuracy: 0.3959 - val_loss: 1.3348 - val_accuracy: 0.4939
Epoch 6/20
17/17 [==============================] - 7s 411ms/step - loss: 1.3910 - accuracy: 0.4237 - val_loss: 1.2401 - val_accuracy: 0.5182
Epoch 7/20
17/17 [==============================] - 7s 413ms/step - loss: 1.3451 - accuracy: 0.4741 - val_loss: 1.3251 - val_accuracy: 0.4504
Epo

In [60]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

3
Epoch 1/20
22/22 [==============================] - 13s 400ms/step - loss: 1.7866 - accuracy: 0.1977 - val_loss: 1.5546 - val_accuracy: 0.3442
Epoch 2/20
22/22 [==============================] - 8s 388ms/step - loss: 1.7836 - accuracy: 0.2924 - val_loss: 1.6784 - val_accuracy: 0.3652
Epoch 3/20
22/22 [==============================] - 9s 394ms/step - loss: 1.5658 - accuracy: 0.3556 - val_loss: 1.3253 - val_accuracy: 0.4483
Epoch 4/20
22/22 [==============================] - 9s 395ms/step - loss: 1.4847 - accuracy: 0.3988 - val_loss: 1.2296 - val_accuracy: 0.4936
Epoch 5/20
22/22 [==============================] - 9s 392ms/step - loss: 1.3858 - accuracy: 0.4240 - val_loss: 1.4324 - val_accuracy: 0.4165
Epoch 6/20
22/22 [==============================] - 9s 390ms/step - loss: 1.3295 - accuracy: 0.4668 - val_loss: 1.1332 - val_accuracy: 0.5339
Epoch 7/20
22/22 [==============================] - 9s 391ms/step - loss: 1.2928 - accuracy: 0.4624 - val_loss: 1.0726 - val_accuracy: 0.5660
Epo

In [61]:
print(n)#4

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

4
Epoch 1/20
28/28 [==============================] - 14s 364ms/step - loss: 1.7917 - accuracy: 0.2159 - val_loss: 1.7147 - val_accuracy: 0.3039
Epoch 2/20
28/28 [==============================] - 10s 371ms/step - loss: 1.6381 - accuracy: 0.3287 - val_loss: 1.4179 - val_accuracy: 0.3549
Epoch 3/20
28/28 [==============================] - 10s 368ms/step - loss: 1.4947 - accuracy: 0.3865 - val_loss: 1.3442 - val_accuracy: 0.5039
Epoch 4/20
28/28 [==============================] - 10s 366ms/step - loss: 1.3622 - accuracy: 0.4532 - val_loss: 1.2891 - val_accuracy: 0.4540
Epoch 5/20
28/28 [==============================] - 10s 372ms/step - loss: 1.3043 - accuracy: 0.4647 - val_loss: 1.2726 - val_accuracy: 0.4964
Epoch 6/20
28/28 [==============================] - 10s 367ms/step - loss: 1.2952 - accuracy: 0.5100 - val_loss: 1.1045 - val_accuracy: 0.5745
Epoch 7/20
28/28 [==============================] - 10s 368ms/step - loss: 1.2098 - accuracy: 0.5291 - val_loss: 1.0731 - val_accuracy: 0.57

In [62]:
n=5
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

5
Epoch 1/20
33/33 [==============================] - 16s 356ms/step - loss: 1.7674 - accuracy: 0.2425 - val_loss: 1.6069 - val_accuracy: 0.3762
Epoch 2/20
33/33 [==============================] - 12s 362ms/step - loss: 1.5095 - accuracy: 0.4489 - val_loss: 1.4011 - val_accuracy: 0.3930
Epoch 3/20
33/33 [==============================] - 12s 362ms/step - loss: 1.4061 - accuracy: 0.4510 - val_loss: 1.2930 - val_accuracy: 0.4650
Epoch 4/20
33/33 [==============================] - 12s 360ms/step - loss: 1.2748 - accuracy: 0.5080 - val_loss: 1.5856 - val_accuracy: 0.3641
Epoch 5/20
33/33 [==============================] - 12s 361ms/step - loss: 1.2803 - accuracy: 0.5090 - val_loss: 1.1828 - val_accuracy: 0.5132
Epoch 6/20
33/33 [==============================] - 12s 362ms/step - loss: 1.1752 - accuracy: 0.5524 - val_loss: 1.3295 - val_accuracy: 0.4540
Epoch 7/20
33/33 [==============================] - 12s 366ms/step - loss: 1.1470 - accuracy: 0.5664 - val_loss: 1.2189 - val_accuracy: 0.47

In [63]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

6
Epoch 1/20
39/39 [==============================] - 17s 341ms/step - loss: 1.7618 - accuracy: 0.2799 - val_loss: 1.9996 - val_accuracy: 0.2168
Epoch 2/20
39/39 [==============================] - 14s 353ms/step - loss: 1.5968 - accuracy: 0.3963 - val_loss: 1.7541 - val_accuracy: 0.2899
Epoch 3/20
39/39 [==============================] - 14s 348ms/step - loss: 1.3713 - accuracy: 0.5027 - val_loss: 1.4750 - val_accuracy: 0.4144
Epoch 4/20
39/39 [==============================] - 13s 347ms/step - loss: 1.2793 - accuracy: 0.5267 - val_loss: 1.3028 - val_accuracy: 0.4665
Epoch 5/20
39/39 [==============================] - 14s 348ms/step - loss: 1.1251 - accuracy: 0.5906 - val_loss: 1.1882 - val_accuracy: 0.5061
Epoch 6/20
39/39 [==============================] - 14s 372ms/step - loss: 1.0769 - accuracy: 0.6016 - val_loss: 1.9262 - val_accuracy: 0.3499
Epoch 7/20
39/39 [==============================] - 14s 355ms/step - loss: 1.1023 - accuracy: 0.5924 - val_loss: 1.1275 - val_accuracy: 0.54

In [64]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

7
Epoch 1/20
44/44 [==============================] - 19s 341ms/step - loss: 1.6782 - accuracy: 0.3632 - val_loss: 1.6227 - val_accuracy: 0.3006
Epoch 2/20
44/44 [==============================] - 15s 348ms/step - loss: 1.4081 - accuracy: 0.5085 - val_loss: 1.4283 - val_accuracy: 0.4055
Epoch 3/20
44/44 [==============================] - 15s 349ms/step - loss: 1.2608 - accuracy: 0.5354 - val_loss: 1.4570 - val_accuracy: 0.3862
Epoch 4/20
44/44 [==============================] - 15s 347ms/step - loss: 1.1962 - accuracy: 0.5793 - val_loss: 1.3703 - val_accuracy: 0.4611
Epoch 5/20
44/44 [==============================] - 15s 344ms/step - loss: 1.1011 - accuracy: 0.6024 - val_loss: 1.1581 - val_accuracy: 0.5157
Epoch 6/20
44/44 [==============================] - 15s 347ms/step - loss: 1.0249 - accuracy: 0.6366 - val_loss: 1.0844 - val_accuracy: 0.5456
Epoch 7/20
44/44 [==============================] - 15s 347ms/step - loss: 0.9627 - accuracy: 0.6579 - val_loss: 1.0098 - val_accuracy: 0.57

In [65]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

8
Epoch 1/20
50/50 [==============================] - 21s 333ms/step - loss: 1.7281 - accuracy: 0.3702 - val_loss: 1.8111 - val_accuracy: 0.2932
Epoch 2/20
50/50 [==============================] - 17s 339ms/step - loss: 1.3416 - accuracy: 0.5104 - val_loss: 1.5928 - val_accuracy: 0.3377
Epoch 3/20
50/50 [==============================] - 17s 344ms/step - loss: 1.2213 - accuracy: 0.5527 - val_loss: 1.3694 - val_accuracy: 0.4447
Epoch 4/20
50/50 [==============================] - 17s 338ms/step - loss: 1.1057 - accuracy: 0.6063 - val_loss: 1.4164 - val_accuracy: 0.4069
Epoch 5/20
50/50 [==============================] - 17s 338ms/step - loss: 1.0400 - accuracy: 0.6197 - val_loss: 1.0391 - val_accuracy: 0.5817
Epoch 6/20
50/50 [==============================] - 17s 340ms/step - loss: 0.9670 - accuracy: 0.6658 - val_loss: 1.1502 - val_accuracy: 0.5193
Epoch 7/20
50/50 [==============================] - 17s 348ms/step - loss: 0.9186 - accuracy: 0.6769 - val_loss: 1.0301 - val_accuracy: 0.58

In [66]:
print(n)#9

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

9
Epoch 1/20
55/55 [==============================] - 23s 339ms/step - loss: 1.7375 - accuracy: 0.3237 - val_loss: 1.5999 - val_accuracy: 0.3310
Epoch 2/20
55/55 [==============================] - 18s 338ms/step - loss: 1.3789 - accuracy: 0.4890 - val_loss: 1.4171 - val_accuracy: 0.4098
Epoch 3/20
55/55 [==============================] - 18s 337ms/step - loss: 1.2209 - accuracy: 0.5599 - val_loss: 1.3584 - val_accuracy: 0.4333
Epoch 4/20
55/55 [==============================] - 18s 337ms/step - loss: 1.1583 - accuracy: 0.5888 - val_loss: 1.2213 - val_accuracy: 0.4675
Epoch 5/20
55/55 [==============================] - 18s 337ms/step - loss: 1.0363 - accuracy: 0.6344 - val_loss: 1.1197 - val_accuracy: 0.5435
Epoch 6/20
55/55 [==============================] - 19s 341ms/step - loss: 0.9831 - accuracy: 0.6584 - val_loss: 1.1288 - val_accuracy: 0.5307
Epoch 7/20
55/55 [==============================] - 18s 336ms/step - loss: 0.9280 - accuracy: 0.6633 - val_loss: 0.9923 - val_accuracy: 0.60

In [67]:
n=10
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

10
Epoch 1/20
61/61 [==============================] - 24s 328ms/step - loss: 1.6588 - accuracy: 0.3122 - val_loss: 1.6133 - val_accuracy: 0.3549
Epoch 2/20
61/61 [==============================] - 20s 331ms/step - loss: 1.2653 - accuracy: 0.5401 - val_loss: 2.3746 - val_accuracy: 0.2885
Epoch 3/20
61/61 [==============================] - 20s 330ms/step - loss: 1.2305 - accuracy: 0.5712 - val_loss: 1.2165 - val_accuracy: 0.4932
Epoch 4/20
61/61 [==============================] - 20s 330ms/step - loss: 1.0392 - accuracy: 0.6404 - val_loss: 1.9456 - val_accuracy: 0.2967
Epoch 5/20
61/61 [==============================] - 20s 332ms/step - loss: 1.0158 - accuracy: 0.6498 - val_loss: 1.0807 - val_accuracy: 0.5685
Epoch 6/20
61/61 [==============================] - 20s 331ms/step - loss: 0.9073 - accuracy: 0.6784 - val_loss: 1.0210 - val_accuracy: 0.5695
Epoch 7/20
61/61 [==============================] - 20s 330ms/step - loss: 0.8571 - accuracy: 0.6986 - val_loss: 1.1774 - val_accuracy: 0.5

In [68]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

11
Epoch 1/20
66/66 [==============================] - 29s 359ms/step - loss: 1.6598 - accuracy: 0.3171 - val_loss: 1.5411 - val_accuracy: 0.3684
Epoch 2/20
66/66 [==============================] - 23s 342ms/step - loss: 1.2521 - accuracy: 0.5493 - val_loss: 1.3690 - val_accuracy: 0.4066
Epoch 3/20
66/66 [==============================] - 22s 340ms/step - loss: 1.1188 - accuracy: 0.6048 - val_loss: 1.2662 - val_accuracy: 0.4815
Epoch 4/20
66/66 [==============================] - 22s 330ms/step - loss: 0.9875 - accuracy: 0.6510 - val_loss: 1.1902 - val_accuracy: 0.5139
Epoch 5/20
66/66 [==============================] - 23s 342ms/step - loss: 0.9149 - accuracy: 0.6804 - val_loss: 1.1896 - val_accuracy: 0.5221
Epoch 6/20
66/66 [==============================] - 23s 342ms/step - loss: 0.8497 - accuracy: 0.6993 - val_loss: 1.1073 - val_accuracy: 0.5517
Epoch 7/20
66/66 [==============================] - 23s 342ms/step - loss: 0.8057 - accuracy: 0.7206 - val_loss: 0.9801 - val_accuracy: 0.5

In [69]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

12
Epoch 1/20
72/72 [==============================] - 28s 326ms/step - loss: 1.6609 - accuracy: 0.3100 - val_loss: 1.4194 - val_accuracy: 0.3834
Epoch 2/20
72/72 [==============================] - 24s 327ms/step - loss: 1.2510 - accuracy: 0.5467 - val_loss: 1.6403 - val_accuracy: 0.3103
Epoch 3/20
72/72 [==============================] - 24s 329ms/step - loss: 1.1440 - accuracy: 0.5862 - val_loss: 1.7953 - val_accuracy: 0.3598
Epoch 4/20
72/72 [==============================] - 24s 330ms/step - loss: 1.0228 - accuracy: 0.6321 - val_loss: 2.0011 - val_accuracy: 0.3324
Epoch 5/20
72/72 [==============================] - 24s 328ms/step - loss: 1.0306 - accuracy: 0.6418 - val_loss: 1.1255 - val_accuracy: 0.5499
Epoch 6/20
72/72 [==============================] - 24s 327ms/step - loss: 0.8774 - accuracy: 0.6891 - val_loss: 1.0173 - val_accuracy: 0.5956
Epoch 7/20
72/72 [==============================] - 23s 326ms/step - loss: 0.8321 - accuracy: 0.6968 - val_loss: 1.0095 - val_accuracy: 0.6

In [70]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

13
Epoch 1/20
77/77 [==============================] - 29s 327ms/step - loss: 1.6522 - accuracy: 0.3261 - val_loss: 1.4512 - val_accuracy: 0.3809
Epoch 2/20
77/77 [==============================] - 25s 328ms/step - loss: 1.2884 - accuracy: 0.5039 - val_loss: 1.3189 - val_accuracy: 0.4344
Epoch 3/20
77/77 [==============================] - 25s 330ms/step - loss: 1.1288 - accuracy: 0.5690 - val_loss: 1.1134 - val_accuracy: 0.5410
Epoch 4/20
77/77 [==============================] - 26s 333ms/step - loss: 1.0573 - accuracy: 0.6040 - val_loss: 1.0214 - val_accuracy: 0.5795
Epoch 5/20
77/77 [==============================] - 25s 329ms/step - loss: 0.9296 - accuracy: 0.6554 - val_loss: 1.1466 - val_accuracy: 0.5581
Epoch 6/20
77/77 [==============================] - 25s 327ms/step - loss: 0.9441 - accuracy: 0.6539 - val_loss: 0.8516 - val_accuracy: 0.6780
Epoch 7/20
77/77 [==============================] - 25s 328ms/step - loss: 0.8292 - accuracy: 0.7008 - val_loss: 1.0240 - val_accuracy: 0.5

In [71]:
print(n)#14

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

14
Epoch 1/20
83/83 [==============================] - 31s 323ms/step - loss: 1.6986 - accuracy: 0.2760 - val_loss: 1.4540 - val_accuracy: 0.3762
Epoch 2/20
83/83 [==============================] - 27s 326ms/step - loss: 1.2892 - accuracy: 0.4927 - val_loss: 1.2241 - val_accuracy: 0.4882
Epoch 3/20
83/83 [==============================] - 27s 325ms/step - loss: 1.1106 - accuracy: 0.5639 - val_loss: 1.0511 - val_accuracy: 0.5317
Epoch 4/20
83/83 [==============================] - 27s 327ms/step - loss: 1.0361 - accuracy: 0.5984 - val_loss: 1.7767 - val_accuracy: 0.3834
Epoch 5/20
83/83 [==============================] - 27s 325ms/step - loss: 0.9730 - accuracy: 0.6389 - val_loss: 1.0349 - val_accuracy: 0.5817
Epoch 6/20
83/83 [==============================] - 27s 326ms/step - loss: 0.8499 - accuracy: 0.6901 - val_loss: 1.0153 - val_accuracy: 0.5942
Epoch 7/20
83/83 [==============================] - 27s 327ms/step - loss: 0.8283 - accuracy: 0.6935 - val_loss: 1.0468 - val_accuracy: 0.5

In [72]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

15
Epoch 1/20
88/88 [==============================] - 33s 325ms/step - loss: 1.7162 - accuracy: 0.2677 - val_loss: 1.4080 - val_accuracy: 0.4640
Epoch 2/20
88/88 [==============================] - 29s 326ms/step - loss: 1.2647 - accuracy: 0.5071 - val_loss: 1.5853 - val_accuracy: 0.3713
Epoch 3/20
88/88 [==============================] - 29s 324ms/step - loss: 1.1036 - accuracy: 0.5650 - val_loss: 1.3582 - val_accuracy: 0.4722
Epoch 4/20
88/88 [==============================] - 29s 327ms/step - loss: 1.0198 - accuracy: 0.6103 - val_loss: 1.1751 - val_accuracy: 0.5724
Epoch 5/20
88/88 [==============================] - 29s 324ms/step - loss: 0.9402 - accuracy: 0.6453 - val_loss: 0.8813 - val_accuracy: 0.6330
Epoch 6/20
88/88 [==============================] - 29s 325ms/step - loss: 0.8427 - accuracy: 0.6910 - val_loss: 0.8767 - val_accuracy: 0.6569
Epoch 7/20
88/88 [==============================] - 28s 324ms/step - loss: 0.8087 - accuracy: 0.6990 - val_loss: 0.8294 - val_accuracy: 0.6

In [73]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

16
Epoch 1/20
93/93 [==============================] - 36s 329ms/step - loss: 1.6705 - accuracy: 0.2819 - val_loss: 1.3272 - val_accuracy: 0.4379
Epoch 2/20
93/93 [==============================] - 30s 326ms/step - loss: 1.2692 - accuracy: 0.4968 - val_loss: 1.0158 - val_accuracy: 0.6198
Epoch 3/20
93/93 [==============================] - 30s 326ms/step - loss: 1.0661 - accuracy: 0.5824 - val_loss: 0.9223 - val_accuracy: 0.6505
Epoch 4/20
93/93 [==============================] - 31s 330ms/step - loss: 1.0047 - accuracy: 0.6162 - val_loss: 0.9385 - val_accuracy: 0.6120
Epoch 5/20
93/93 [==============================] - 30s 326ms/step - loss: 0.9186 - accuracy: 0.6613 - val_loss: 0.8483 - val_accuracy: 0.6651
Epoch 6/20
93/93 [==============================] - 30s 325ms/step - loss: 0.8871 - accuracy: 0.6664 - val_loss: 0.8007 - val_accuracy: 0.6926
Epoch 7/20
93/93 [==============================] - 30s 326ms/step - loss: 0.8248 - accuracy: 0.6902 - val_loss: 0.9870 - val_accuracy: 0.5

In [74]:
time.sleep(400)

In [75]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

17
Epoch 1/20
99/99 [==============================] - 34s 299ms/step - loss: 1.7344 - accuracy: 0.2633 - val_loss: 1.5606 - val_accuracy: 0.3663
Epoch 2/20
99/99 [==============================] - 29s 292ms/step - loss: 1.2892 - accuracy: 0.4970 - val_loss: 1.0696 - val_accuracy: 0.5777
Epoch 3/20
99/99 [==============================] - 29s 294ms/step - loss: 1.0944 - accuracy: 0.5745 - val_loss: 1.0222 - val_accuracy: 0.5859
Epoch 4/20
99/99 [==============================] - 29s 294ms/step - loss: 1.0347 - accuracy: 0.6058 - val_loss: 0.9272 - val_accuracy: 0.6466
Epoch 5/20
99/99 [==============================] - 29s 289ms/step - loss: 0.9484 - accuracy: 0.6444 - val_loss: 0.8251 - val_accuracy: 0.6858
Epoch 6/20
99/99 [==============================] - 29s 295ms/step - loss: 0.8739 - accuracy: 0.6719 - val_loss: 1.1428 - val_accuracy: 0.5524
Epoch 7/20
99/99 [==============================] - 29s 294ms/step - loss: 0.8497 - accuracy: 0.6802 - val_loss: 0.7579 - val_accuracy: 0.7

In [76]:
time.sleep(400)

In [77]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

18
Epoch 1/20
104/104 [==============================] - 35s 298ms/step - loss: 1.6972 - accuracy: 0.2701 - val_loss: 1.2719 - val_accuracy: 0.5039
Epoch 2/20
104/104 [==============================] - 31s 295ms/step - loss: 1.2569 - accuracy: 0.5083 - val_loss: 1.2339 - val_accuracy: 0.5185
Epoch 3/20
104/104 [==============================] - 31s 294ms/step - loss: 1.1105 - accuracy: 0.5668 - val_loss: 1.0148 - val_accuracy: 0.5938
Epoch 4/20
104/104 [==============================] - 31s 297ms/step - loss: 1.0477 - accuracy: 0.5943 - val_loss: 0.9348 - val_accuracy: 0.6298
Epoch 5/20
104/104 [==============================] - 31s 294ms/step - loss: 0.9507 - accuracy: 0.6429 - val_loss: 0.7982 - val_accuracy: 0.6879
Epoch 6/20
104/104 [==============================] - 31s 294ms/step - loss: 0.8761 - accuracy: 0.6740 - val_loss: 0.7866 - val_accuracy: 0.7033
Epoch 7/20
104/104 [==============================] - 31s 294ms/step - loss: 0.8046 - accuracy: 0.7030 - val_loss: 0.6985 - val

In [78]:
time.sleep(400)

In [79]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

19
Epoch 1/20
110/110 [==============================] - 37s 294ms/step - loss: 1.6352 - accuracy: 0.2959 - val_loss: 1.3420 - val_accuracy: 0.4618
Epoch 2/20
110/110 [==============================] - 32s 290ms/step - loss: 1.2440 - accuracy: 0.5048 - val_loss: 0.9576 - val_accuracy: 0.6245
Epoch 3/20
110/110 [==============================] - 32s 290ms/step - loss: 1.0850 - accuracy: 0.5718 - val_loss: 1.1452 - val_accuracy: 0.5371
Epoch 4/20
110/110 [==============================] - 32s 291ms/step - loss: 1.0161 - accuracy: 0.6130 - val_loss: 1.1561 - val_accuracy: 0.5342
Epoch 5/20
110/110 [==============================] - 32s 290ms/step - loss: 0.9115 - accuracy: 0.6520 - val_loss: 0.8227 - val_accuracy: 0.6776
Epoch 6/20
110/110 [==============================] - 32s 290ms/step - loss: 0.8625 - accuracy: 0.6763 - val_loss: 0.7272 - val_accuracy: 0.7279
Epoch 7/20
110/110 [==============================] - 32s 290ms/step - loss: 0.7806 - accuracy: 0.7089 - val_loss: 0.6755 - val

In [80]:
time.sleep(400)

In [81]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

20
Epoch 1/20
112/112 [==============================] - 37s 295ms/step - loss: 1.6944 - accuracy: 0.2655 - val_loss: 1.2557 - val_accuracy: 0.5118
Epoch 2/20
112/112 [==============================] - 32s 289ms/step - loss: 1.2443 - accuracy: 0.5006 - val_loss: 1.6288 - val_accuracy: 0.3652
Epoch 3/20
112/112 [==============================] - 33s 291ms/step - loss: 1.1188 - accuracy: 0.5571 - val_loss: 1.1920 - val_accuracy: 0.4964
Epoch 4/20
112/112 [==============================] - 33s 292ms/step - loss: 1.0174 - accuracy: 0.6056 - val_loss: 1.0132 - val_accuracy: 0.5895
Epoch 5/20
112/112 [==============================] - 33s 291ms/step - loss: 0.9285 - accuracy: 0.6437 - val_loss: 0.8472 - val_accuracy: 0.6837
Epoch 6/20
112/112 [==============================] - 33s 291ms/step - loss: 0.8713 - accuracy: 0.6779 - val_loss: 0.8775 - val_accuracy: 0.6655
Epoch 7/20
112/112 [==============================] - 33s 291ms/step - loss: 0.7902 - accuracy: 0.7071 - val_loss: 0.7269 - val

In [82]:
time.sleep(400)

In [84]:
new_model_dsa_dir  = "D:/models/aug_22/"+dataset+"/C1/"+dataset+"_model_c1_aug_dsa_e1"

i=0

for model in models_dsa:
    model.save(new_model_dsa_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_dsa_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_dsa_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_dsa_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_dsa_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_dsa_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_dsa_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_dsa_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_dsa_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/int

In [ ]:
loading = False

models_dsa = []

if loading:
    for i in range(20):
        model_dsa_dir = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_dsa_e2_"+str(i)
        print(model_dsa_dir)
        model =utils.My_model('gtsrb',True,model_dsa_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_dsa.append(model)
       
    

In [85]:
try:
    del dsa_values
    del top_images_by_dsa
    del top_labels_by_dsa
    del image_sets_dsa
    del label_sets_dsa
    del models_dsa
except:
    print("Error")

In [86]:
gc.collect()

60838

## Uncertainty

In [87]:
dg_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/deep_gini_values.npy"

deep_gini_values = np.load(dg_direction)


In [88]:
# Obtaining top n images by dg values
top_images_by_dg  = utils.get_x_of_indexes(list(np.flip(np.argsort(deep_gini_values))),x_train_and_adversary)
top_labels_by_dg = utils.get_x_of_indexes(list(np.flip(np.argsort(deep_gini_values))),y_train_and_adversary)

In [89]:
m = n_data_points
n = 0
image_sets_dg = []
label_sets_dg = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_dg)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_dg)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_dg)%m))
        top_images_by_dg_n = np.array(top_images_by_dg[:n+m+(len(top_images_by_dg)%m)])
        top_labels_by_dg_n = np.array(top_labels_by_dg[:n+m+(len(top_images_by_dg)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_dg_n = np.array(top_images_by_dg[:n+m])
        top_labels_by_dg_n = np.array(top_labels_by_dg[:n+m])
    image_sets_dg.append(top_images_by_dg_n)
    label_sets_dg.append(top_labels_by_dg_n)
    print(len(top_images_by_dg_n))
    n += m



0 :
0  ->  700
700
1 :
0  ->  1400
1400
2 :
0  ->  2100
2100
3 :
0  ->  2800
2800
4 :
0  ->  3500
3500
5 :
0  ->  4200
4200
6 :
0  ->  4900
4900
7 :
0  ->  5600
5600
8 :
0  ->  6300
6300
9 :
0  ->  7000
7000
10 :
0  ->  7700
7700
11 :
0  ->  8400
8400
12 :
0  ->  9100
9100
13 :
0  ->  9800
9800
14 :
0  ->  10500
10500
15 :
0  ->  11200
11200
16 :
0  ->  11900
11900
17 :
0  ->  12600
12600
18 :
0  ->  13300
13300
19 :
0  ->  14000
14000
20 :
Last
0  ->  14924
14224


In [90]:
models_dg = []
#for i in range(len(label_sets_lsa)):
for i in range(len(label_sets_dg)):
    print(i,":")
    model = utils.My_model(dataset,False)
    model.compile_model()
    models_dg.append(model)

0 :
Model compiled
1 :
Model compiled
2 :
Model compiled
3 :
Model compiled
4 :
Model compiled
5 :
Model compiled
6 :
Model compiled
7 :
Model compiled
8 :
Model compiled
9 :
Model compiled
10 :
Model compiled
11 :
Model compiled
12 :
Model compiled
13 :
Model compiled
14 :
Model compiled
15 :
Model compiled
16 :
Model compiled
17 :
Model compiled
18 :
Model compiled
19 :
Model compiled
20 :
Model compiled


In [91]:
n=0
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

0
Epoch 1/20
6/6 [==============================] - 6s 597ms/step - loss: 1.8060 - accuracy: 0.2022 - val_loss: 1.7929 - val_accuracy: 0.1619
Epoch 2/20
6/6 [==============================] - 3s 488ms/step - loss: 1.7592 - accuracy: 0.2320 - val_loss: 1.8572 - val_accuracy: 0.1790
Epoch 3/20
6/6 [==============================] - 3s 498ms/step - loss: 1.7578 - accuracy: 0.2599 - val_loss: 1.7998 - val_accuracy: 0.1790
Epoch 4/20
6/6 [==============================] - 3s 499ms/step - loss: 1.7161 - accuracy: 0.2779 - val_loss: 1.7903 - val_accuracy: 0.1790
Epoch 5/20
6/6 [==============================] - 3s 506ms/step - loss: 1.7678 - accuracy: 0.2554 - val_loss: 1.7917 - val_accuracy: 0.1790
Epoch 6/20
6/6 [==============================] - 3s 506ms/step - loss: 1.7222 - accuracy: 0.2660 - val_loss: 1.7728 - val_accuracy: 0.1790
Epoch 7/20
6/6 [==============================] - 3s 508ms/step - loss: 1.7038 - accuracy: 0.2709 - val_loss: 1.7340 - val_accuracy: 0.2161
Epoch 8/20
6/6 [==

In [92]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

1
Epoch 1/20
11/11 [==============================] - 8s 437ms/step - loss: 1.7747 - accuracy: 0.2329 - val_loss: 1.7995 - val_accuracy: 0.1790
Epoch 2/20
11/11 [==============================] - 4s 387ms/step - loss: 1.7345 - accuracy: 0.2638 - val_loss: 1.8375 - val_accuracy: 0.1790
Epoch 3/20
11/11 [==============================] - 4s 394ms/step - loss: 1.7315 - accuracy: 0.2714 - val_loss: 1.7356 - val_accuracy: 0.1790
Epoch 4/20
11/11 [==============================] - 4s 397ms/step - loss: 1.6956 - accuracy: 0.2738 - val_loss: 1.6873 - val_accuracy: 0.1901
Epoch 5/20
11/11 [==============================] - 4s 390ms/step - loss: 1.6758 - accuracy: 0.2880 - val_loss: 1.6571 - val_accuracy: 0.3092
Epoch 6/20
11/11 [==============================] - 4s 394ms/step - loss: 1.6682 - accuracy: 0.2997 - val_loss: 1.6539 - val_accuracy: 0.3659
Epoch 7/20
11/11 [==============================] - 4s 391ms/step - loss: 1.6494 - accuracy: 0.3216 - val_loss: 1.6188 - val_accuracy: 0.3509
Epoc

In [93]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

2
Epoch 1/20
17/17 [==============================] - 9s 359ms/step - loss: 1.7555 - accuracy: 0.2457 - val_loss: 1.8123 - val_accuracy: 0.1790
Epoch 2/20
17/17 [==============================] - 5s 327ms/step - loss: 1.6796 - accuracy: 0.3012 - val_loss: 1.7554 - val_accuracy: 0.1819
Epoch 3/20
17/17 [==============================] - 5s 327ms/step - loss: 1.6315 - accuracy: 0.3137 - val_loss: 1.6057 - val_accuracy: 0.3131
Epoch 4/20
17/17 [==============================] - 6s 328ms/step - loss: 1.6441 - accuracy: 0.2825 - val_loss: 1.5443 - val_accuracy: 0.3402
Epoch 5/20
17/17 [==============================] - 6s 329ms/step - loss: 1.5964 - accuracy: 0.3543 - val_loss: 1.5075 - val_accuracy: 0.3702
Epoch 6/20
17/17 [==============================] - 5s 327ms/step - loss: 1.5870 - accuracy: 0.3405 - val_loss: 1.6255 - val_accuracy: 0.2935
Epoch 7/20
17/17 [==============================] - 6s 330ms/step - loss: 1.5820 - accuracy: 0.3445 - val_loss: 1.4779 - val_accuracy: 0.3213
Epoc

In [94]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

3
Epoch 1/20
22/22 [==============================] - 11s 349ms/step - loss: 1.8505 - accuracy: 0.2156 - val_loss: 1.8442 - val_accuracy: 0.1790
Epoch 2/20
22/22 [==============================] - 7s 317ms/step - loss: 1.6548 - accuracy: 0.2896 - val_loss: 1.7122 - val_accuracy: 0.1797
Epoch 3/20
22/22 [==============================] - 7s 329ms/step - loss: 1.6052 - accuracy: 0.3390 - val_loss: 1.5378 - val_accuracy: 0.3477
Epoch 4/20
22/22 [==============================] - 7s 332ms/step - loss: 1.5766 - accuracy: 0.3426 - val_loss: 1.4433 - val_accuracy: 0.3969
Epoch 5/20
22/22 [==============================] - 7s 323ms/step - loss: 1.5827 - accuracy: 0.3244 - val_loss: 1.5326 - val_accuracy: 0.3335
Epoch 6/20
22/22 [==============================] - 7s 318ms/step - loss: 1.5452 - accuracy: 0.3542 - val_loss: 1.5240 - val_accuracy: 0.3755
Epoch 7/20
22/22 [==============================] - 7s 327ms/step - loss: 1.5057 - accuracy: 0.3802 - val_loss: 1.4472 - val_accuracy: 0.3923
Epo

In [95]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

4
Epoch 1/20
28/28 [==============================] - 12s 314ms/step - loss: 1.7550 - accuracy: 0.2257 - val_loss: 1.7457 - val_accuracy: 0.1794
Epoch 2/20
28/28 [==============================] - 8s 298ms/step - loss: 1.6259 - accuracy: 0.3271 - val_loss: 1.6369 - val_accuracy: 0.2807
Epoch 3/20
28/28 [==============================] - 9s 328ms/step - loss: 1.5648 - accuracy: 0.3460 - val_loss: 1.4367 - val_accuracy: 0.3745
Epoch 4/20
28/28 [==============================] - 9s 325ms/step - loss: 1.5603 - accuracy: 0.3579 - val_loss: 1.4322 - val_accuracy: 0.3367
Epoch 5/20
28/28 [==============================] - 9s 324ms/step - loss: 1.5319 - accuracy: 0.3568 - val_loss: 1.4483 - val_accuracy: 0.3716
Epoch 6/20
28/28 [==============================] - 9s 323ms/step - loss: 1.5037 - accuracy: 0.3772 - val_loss: 1.4225 - val_accuracy: 0.3738
Epoch 7/20
28/28 [==============================] - 9s 325ms/step - loss: 1.4862 - accuracy: 0.3853 - val_loss: 1.4107 - val_accuracy: 0.4148
Epo

In [96]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

5
Epoch 1/20
33/33 [==============================] - 13s 315ms/step - loss: 1.7388 - accuracy: 0.2540 - val_loss: 1.6866 - val_accuracy: 0.2422
Epoch 2/20
33/33 [==============================] - 11s 344ms/step - loss: 1.6003 - accuracy: 0.3283 - val_loss: 1.5248 - val_accuracy: 0.3356
Epoch 3/20
33/33 [==============================] - 11s 323ms/step - loss: 1.5386 - accuracy: 0.3677 - val_loss: 1.5324 - val_accuracy: 0.3409
Epoch 4/20
33/33 [==============================] - 11s 325ms/step - loss: 1.5054 - accuracy: 0.3603 - val_loss: 1.3989 - val_accuracy: 0.3620
Epoch 5/20
33/33 [==============================] - 11s 323ms/step - loss: 1.4847 - accuracy: 0.3868 - val_loss: 1.2845 - val_accuracy: 0.4344
Epoch 6/20
33/33 [==============================] - 11s 333ms/step - loss: 1.4789 - accuracy: 0.3908 - val_loss: 1.3811 - val_accuracy: 0.3837
Epoch 7/20
33/33 [==============================] - 11s 332ms/step - loss: 1.4139 - accuracy: 0.4090 - val_loss: 1.3557 - val_accuracy: 0.37

In [97]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

6
Epoch 1/20
39/39 [==============================] - 16s 314ms/step - loss: 1.7133 - accuracy: 0.2638 - val_loss: 1.7067 - val_accuracy: 0.2660
Epoch 2/20
39/39 [==============================] - 12s 320ms/step - loss: 1.6035 - accuracy: 0.3481 - val_loss: 1.5348 - val_accuracy: 0.3834
Epoch 3/20
39/39 [==============================] - 12s 320ms/step - loss: 1.5118 - accuracy: 0.3815 - val_loss: 2.1562 - val_accuracy: 0.2454
Epoch 4/20
39/39 [==============================] - 12s 320ms/step - loss: 1.5253 - accuracy: 0.3786 - val_loss: 1.6335 - val_accuracy: 0.3499
Epoch 5/20
39/39 [==============================] - 12s 319ms/step - loss: 1.4304 - accuracy: 0.4019 - val_loss: 1.3455 - val_accuracy: 0.4048
Epoch 6/20
39/39 [==============================] - 12s 318ms/step - loss: 1.4016 - accuracy: 0.4131 - val_loss: 1.2747 - val_accuracy: 0.4116
Epoch 7/20
39/39 [==============================] - 12s 319ms/step - loss: 1.3696 - accuracy: 0.4225 - val_loss: 1.1946 - val_accuracy: 0.49

In [98]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

7
Epoch 1/20
44/44 [==============================] - 17s 317ms/step - loss: 1.6985 - accuracy: 0.2570 - val_loss: 1.5332 - val_accuracy: 0.3377
Epoch 2/20
44/44 [==============================] - 14s 318ms/step - loss: 1.5315 - accuracy: 0.3667 - val_loss: 1.4055 - val_accuracy: 0.3887
Epoch 3/20
44/44 [==============================] - 14s 315ms/step - loss: 1.4560 - accuracy: 0.3942 - val_loss: 1.3594 - val_accuracy: 0.4187
Epoch 4/20
44/44 [==============================] - 14s 316ms/step - loss: 1.4351 - accuracy: 0.4022 - val_loss: 1.3930 - val_accuracy: 0.4155
Epoch 5/20
44/44 [==============================] - 14s 317ms/step - loss: 1.3792 - accuracy: 0.4179 - val_loss: 1.4027 - val_accuracy: 0.3859
Epoch 6/20
44/44 [==============================] - 14s 316ms/step - loss: 1.3644 - accuracy: 0.4291 - val_loss: 1.2141 - val_accuracy: 0.4736
Epoch 7/20
44/44 [==============================] - 14s 320ms/step - loss: 1.3363 - accuracy: 0.4375 - val_loss: 1.2540 - val_accuracy: 0.48

In [99]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

8
Epoch 1/20
50/50 [==============================] - 18s 309ms/step - loss: 1.7068 - accuracy: 0.2624 - val_loss: 1.5951 - val_accuracy: 0.3499
Epoch 2/20
50/50 [==============================] - 16s 311ms/step - loss: 1.5756 - accuracy: 0.3625 - val_loss: 1.4743 - val_accuracy: 0.3591
Epoch 3/20
50/50 [==============================] - 16s 316ms/step - loss: 1.4780 - accuracy: 0.3914 - val_loss: 1.4426 - val_accuracy: 0.3773
Epoch 4/20
50/50 [==============================] - 16s 317ms/step - loss: 1.4311 - accuracy: 0.4086 - val_loss: 1.3250 - val_accuracy: 0.4319
Epoch 5/20
50/50 [==============================] - 15s 310ms/step - loss: 1.3969 - accuracy: 0.4255 - val_loss: 1.3907 - val_accuracy: 0.3598
Epoch 6/20
50/50 [==============================] - 16s 311ms/step - loss: 1.3551 - accuracy: 0.4225 - val_loss: 1.1999 - val_accuracy: 0.4636
Epoch 7/20
50/50 [==============================] - 16s 311ms/step - loss: 1.3019 - accuracy: 0.4637 - val_loss: 1.1978 - val_accuracy: 0.46

In [100]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

9
Epoch 1/20
55/55 [==============================] - 23s 344ms/step - loss: 1.6866 - accuracy: 0.2790 - val_loss: 1.5156 - val_accuracy: 0.3424
Epoch 2/20
55/55 [==============================] - 18s 337ms/step - loss: 1.4918 - accuracy: 0.3672 - val_loss: 1.5858 - val_accuracy: 0.3317
Epoch 3/20
55/55 [==============================] - 18s 337ms/step - loss: 1.4247 - accuracy: 0.4063 - val_loss: 1.3230 - val_accuracy: 0.4183
Epoch 4/20
55/55 [==============================] - 19s 339ms/step - loss: 1.3583 - accuracy: 0.4208 - val_loss: 1.4418 - val_accuracy: 0.4151
Epoch 5/20
55/55 [==============================] - 19s 338ms/step - loss: 1.3113 - accuracy: 0.4397 - val_loss: 1.1817 - val_accuracy: 0.4818
Epoch 6/20
55/55 [==============================] - 19s 338ms/step - loss: 1.2736 - accuracy: 0.4648 - val_loss: 1.1703 - val_accuracy: 0.5136
Epoch 7/20
55/55 [==============================] - 19s 341ms/step - loss: 1.2317 - accuracy: 0.4947 - val_loss: 1.1632 - val_accuracy: 0.53

In [101]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

10
Epoch 1/20
61/61 [==============================] - 24s 331ms/step - loss: 1.6852 - accuracy: 0.2664 - val_loss: 1.7336 - val_accuracy: 0.2429
Epoch 2/20
61/61 [==============================] - 20s 332ms/step - loss: 1.5156 - accuracy: 0.3675 - val_loss: 1.4183 - val_accuracy: 0.4108
Epoch 3/20
61/61 [==============================] - 21s 343ms/step - loss: 1.4212 - accuracy: 0.4169 - val_loss: 1.2484 - val_accuracy: 0.4658
Epoch 4/20
61/61 [==============================] - 20s 334ms/step - loss: 1.3422 - accuracy: 0.4403 - val_loss: 1.1597 - val_accuracy: 0.4668
Epoch 5/20
61/61 [==============================] - 20s 333ms/step - loss: 1.2956 - accuracy: 0.4713 - val_loss: 1.2892 - val_accuracy: 0.4469
Epoch 6/20
61/61 [==============================] - 20s 335ms/step - loss: 1.2361 - accuracy: 0.4862 - val_loss: 1.1214 - val_accuracy: 0.5449
Epoch 7/20
61/61 [==============================] - 20s 333ms/step - loss: 1.2035 - accuracy: 0.5046 - val_loss: 1.1043 - val_accuracy: 0.5

In [102]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

11
Epoch 1/20
66/66 [==============================] - 26s 334ms/step - loss: 1.7020 - accuracy: 0.2621 - val_loss: 1.5210 - val_accuracy: 0.4005
Epoch 2/20
66/66 [==============================] - 22s 335ms/step - loss: 1.4797 - accuracy: 0.3929 - val_loss: 1.3553 - val_accuracy: 0.4219
Epoch 3/20
66/66 [==============================] - 22s 335ms/step - loss: 1.3573 - accuracy: 0.4295 - val_loss: 1.2648 - val_accuracy: 0.4622
Epoch 4/20
66/66 [==============================] - 22s 334ms/step - loss: 1.3060 - accuracy: 0.4668 - val_loss: 1.1611 - val_accuracy: 0.4936
Epoch 5/20
66/66 [==============================] - 22s 335ms/step - loss: 1.2518 - accuracy: 0.4794 - val_loss: 1.1743 - val_accuracy: 0.4911
Epoch 6/20
66/66 [==============================] - 23s 348ms/step - loss: 1.2048 - accuracy: 0.5095 - val_loss: 1.0870 - val_accuracy: 0.5250
Epoch 7/20
66/66 [==============================] - 22s 333ms/step - loss: 1.1671 - accuracy: 0.5144 - val_loss: 1.1129 - val_accuracy: 0.5

In [103]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

12
Epoch 1/20
72/72 [==============================] - 28s 328ms/step - loss: 1.6784 - accuracy: 0.2704 - val_loss: 1.5313 - val_accuracy: 0.3288
Epoch 2/20
72/72 [==============================] - 24s 334ms/step - loss: 1.4588 - accuracy: 0.3997 - val_loss: 1.4835 - val_accuracy: 0.3787
Epoch 3/20
72/72 [==============================] - 24s 329ms/step - loss: 1.3603 - accuracy: 0.4365 - val_loss: 1.2665 - val_accuracy: 0.4379
Epoch 4/20
72/72 [==============================] - 24s 330ms/step - loss: 1.2691 - accuracy: 0.4766 - val_loss: 1.0846 - val_accuracy: 0.5670
Epoch 5/20
72/72 [==============================] - 24s 335ms/step - loss: 1.2087 - accuracy: 0.5173 - val_loss: 1.3296 - val_accuracy: 0.3887
Epoch 6/20
72/72 [==============================] - 24s 332ms/step - loss: 1.1853 - accuracy: 0.5220 - val_loss: 1.2509 - val_accuracy: 0.5278
Epoch 7/20
72/72 [==============================] - 24s 333ms/step - loss: 1.1458 - accuracy: 0.5469 - val_loss: 0.9111 - val_accuracy: 0.6

In [104]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

13
Epoch 1/20
77/77 [==============================] - 29s 328ms/step - loss: 1.6909 - accuracy: 0.2621 - val_loss: 1.4385 - val_accuracy: 0.3866
Epoch 2/20
77/77 [==============================] - 25s 330ms/step - loss: 1.4107 - accuracy: 0.4162 - val_loss: 1.2128 - val_accuracy: 0.5050
Epoch 3/20
77/77 [==============================] - 25s 329ms/step - loss: 1.3341 - accuracy: 0.4451 - val_loss: 1.2279 - val_accuracy: 0.4718
Epoch 4/20
77/77 [==============================] - 25s 330ms/step - loss: 1.2297 - accuracy: 0.4931 - val_loss: 1.1049 - val_accuracy: 0.5817
Epoch 5/20
77/77 [==============================] - 26s 335ms/step - loss: 1.1695 - accuracy: 0.5261 - val_loss: 1.1504 - val_accuracy: 0.5578
Epoch 6/20
77/77 [==============================] - 25s 330ms/step - loss: 1.1291 - accuracy: 0.5492 - val_loss: 0.9775 - val_accuracy: 0.6173
Epoch 7/20
77/77 [==============================] - 25s 331ms/step - loss: 1.0851 - accuracy: 0.5668 - val_loss: 0.9119 - val_accuracy: 0.6

In [105]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

14
Epoch 1/20
83/83 [==============================] - 28s 303ms/step - loss: 1.6864 - accuracy: 0.2732 - val_loss: 1.4192 - val_accuracy: 0.3399
Epoch 2/20
83/83 [==============================] - 25s 302ms/step - loss: 1.4181 - accuracy: 0.4118 - val_loss: 1.3284 - val_accuracy: 0.3755
Epoch 3/20
83/83 [==============================] - 25s 304ms/step - loss: 1.2745 - accuracy: 0.4746 - val_loss: 1.2075 - val_accuracy: 0.5250
Epoch 4/20
83/83 [==============================] - 25s 300ms/step - loss: 1.2133 - accuracy: 0.5056 - val_loss: 1.0704 - val_accuracy: 0.5521
Epoch 5/20
83/83 [==============================] - 25s 299ms/step - loss: 1.1754 - accuracy: 0.5266 - val_loss: 1.2478 - val_accuracy: 0.5114
Epoch 6/20
83/83 [==============================] - 25s 301ms/step - loss: 1.1141 - accuracy: 0.5456 - val_loss: 1.6110 - val_accuracy: 0.3345
Epoch 7/20
83/83 [==============================] - 25s 300ms/step - loss: 1.0914 - accuracy: 0.5659 - val_loss: 1.5107 - val_accuracy: 0.4

In [106]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

15
Epoch 1/20
88/88 [==============================] - 29s 299ms/step - loss: 1.6766 - accuracy: 0.2669 - val_loss: 1.5196 - val_accuracy: 0.3499
Epoch 2/20
88/88 [==============================] - 26s 298ms/step - loss: 1.4070 - accuracy: 0.4286 - val_loss: 1.4132 - val_accuracy: 0.4301
Epoch 3/20
88/88 [==============================] - 26s 301ms/step - loss: 1.2407 - accuracy: 0.4980 - val_loss: 1.2314 - val_accuracy: 0.5275
Epoch 4/20
88/88 [==============================] - 26s 298ms/step - loss: 1.1674 - accuracy: 0.5269 - val_loss: 0.9539 - val_accuracy: 0.6519
Epoch 5/20
88/88 [==============================] - 27s 302ms/step - loss: 1.1052 - accuracy: 0.5581 - val_loss: 0.9334 - val_accuracy: 0.6309
Epoch 6/20
88/88 [==============================] - 26s 300ms/step - loss: 1.0572 - accuracy: 0.5872 - val_loss: 1.0004 - val_accuracy: 0.5934
Epoch 7/20
88/88 [==============================] - 26s 299ms/step - loss: 0.9973 - accuracy: 0.6135 - val_loss: 0.8353 - val_accuracy: 0.7

In [107]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

16
Epoch 1/20
93/93 [==============================] - 31s 299ms/step - loss: 1.7119 - accuracy: 0.2579 - val_loss: 1.2441 - val_accuracy: 0.5057
Epoch 2/20
93/93 [==============================] - 28s 300ms/step - loss: 1.3841 - accuracy: 0.4391 - val_loss: 1.1262 - val_accuracy: 0.5374
Epoch 3/20
93/93 [==============================] - 28s 302ms/step - loss: 1.2532 - accuracy: 0.4890 - val_loss: 0.9693 - val_accuracy: 0.6259
Epoch 4/20
93/93 [==============================] - 28s 299ms/step - loss: 1.1391 - accuracy: 0.5499 - val_loss: 0.9254 - val_accuracy: 0.6387
Epoch 5/20
93/93 [==============================] - 29s 309ms/step - loss: 1.0899 - accuracy: 0.5748 - val_loss: 0.8093 - val_accuracy: 0.7058
Epoch 6/20
93/93 [==============================] - 28s 299ms/step - loss: 1.0089 - accuracy: 0.6154 - val_loss: 0.8397 - val_accuracy: 0.6854
Epoch 7/20
93/93 [==============================] - 28s 301ms/step - loss: 0.9891 - accuracy: 0.6235 - val_loss: 0.7553 - val_accuracy: 0.7

In [108]:
time.sleep(400)

In [109]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

17
Epoch 1/20
99/99 [==============================] - 29s 264ms/step - loss: 1.6869 - accuracy: 0.2726 - val_loss: 1.2940 - val_accuracy: 0.4219
Epoch 2/20
99/99 [==============================] - 26s 263ms/step - loss: 1.3318 - accuracy: 0.4569 - val_loss: 1.1190 - val_accuracy: 0.5728
Epoch 3/20
99/99 [==============================] - 26s 263ms/step - loss: 1.1765 - accuracy: 0.5296 - val_loss: 0.9046 - val_accuracy: 0.6566
Epoch 4/20
99/99 [==============================] - 26s 263ms/step - loss: 1.1137 - accuracy: 0.5708 - val_loss: 0.8777 - val_accuracy: 0.6487
Epoch 5/20
99/99 [==============================] - 26s 263ms/step - loss: 1.0097 - accuracy: 0.6068 - val_loss: 0.9272 - val_accuracy: 0.6238
Epoch 6/20
99/99 [==============================] - 26s 262ms/step - loss: 0.9507 - accuracy: 0.6355 - val_loss: 0.7950 - val_accuracy: 0.6986
Epoch 7/20
99/99 [==============================] - 26s 263ms/step - loss: 0.9308 - accuracy: 0.6472 - val_loss: 0.6992 - val_accuracy: 0.7

In [110]:
time.sleep(400)

In [111]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

18
Epoch 1/20
104/104 [==============================] - 30s 266ms/step - loss: 1.6589 - accuracy: 0.2850 - val_loss: 1.1172 - val_accuracy: 0.5656
Epoch 2/20
104/104 [==============================] - 27s 262ms/step - loss: 1.2840 - accuracy: 0.4870 - val_loss: 0.9722 - val_accuracy: 0.6277
Epoch 3/20
104/104 [==============================] - 27s 263ms/step - loss: 1.1140 - accuracy: 0.5626 - val_loss: 1.2031 - val_accuracy: 0.5346
Epoch 4/20
104/104 [==============================] - 27s 263ms/step - loss: 1.0680 - accuracy: 0.5880 - val_loss: 0.8440 - val_accuracy: 0.6822
Epoch 5/20
104/104 [==============================] - 27s 263ms/step - loss: 0.9648 - accuracy: 0.6243 - val_loss: 0.7472 - val_accuracy: 0.7197
Epoch 6/20
104/104 [==============================] - 27s 263ms/step - loss: 0.9220 - accuracy: 0.6514 - val_loss: 0.7661 - val_accuracy: 0.7179
Epoch 7/20
104/104 [==============================] - 27s 263ms/step - loss: 0.8635 - accuracy: 0.6822 - val_loss: 0.8048 - val

In [112]:
time.sleep(400)

In [113]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

19
Epoch 1/20
110/110 [==============================] - 32s 265ms/step - loss: 1.6970 - accuracy: 0.2703 - val_loss: 1.1887 - val_accuracy: 0.5325
Epoch 2/20
110/110 [==============================] - 29s 262ms/step - loss: 1.2832 - accuracy: 0.4895 - val_loss: 1.0674 - val_accuracy: 0.6034
Epoch 3/20
110/110 [==============================] - 29s 262ms/step - loss: 1.1070 - accuracy: 0.5673 - val_loss: 0.8798 - val_accuracy: 0.6851
Epoch 4/20
110/110 [==============================] - 29s 262ms/step - loss: 1.0119 - accuracy: 0.6121 - val_loss: 0.9029 - val_accuracy: 0.6665
Epoch 5/20
110/110 [==============================] - 29s 262ms/step - loss: 0.9473 - accuracy: 0.6472 - val_loss: 1.0517 - val_accuracy: 0.6320
Epoch 6/20
110/110 [==============================] - 29s 262ms/step - loss: 0.8927 - accuracy: 0.6683 - val_loss: 0.7916 - val_accuracy: 0.7044
Epoch 7/20
110/110 [==============================] - 29s 263ms/step - loss: 0.8184 - accuracy: 0.6962 - val_loss: 0.7044 - val

In [114]:
time.sleep(400)

In [115]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

20
Epoch 1/20
112/112 [==============================] - 32s 264ms/step - loss: 1.6938 - accuracy: 0.2738 - val_loss: 1.1581 - val_accuracy: 0.5250
Epoch 2/20
112/112 [==============================] - 29s 262ms/step - loss: 1.2605 - accuracy: 0.5022 - val_loss: 1.0151 - val_accuracy: 0.5927
Epoch 3/20
112/112 [==============================] - 29s 263ms/step - loss: 1.1034 - accuracy: 0.5627 - val_loss: 1.0599 - val_accuracy: 0.6070
Epoch 4/20
112/112 [==============================] - 29s 262ms/step - loss: 1.0066 - accuracy: 0.6074 - val_loss: 1.0481 - val_accuracy: 0.5977
Epoch 5/20
112/112 [==============================] - 29s 261ms/step - loss: 0.9199 - accuracy: 0.6557 - val_loss: 0.8339 - val_accuracy: 0.6872
Epoch 6/20
112/112 [==============================] - 29s 262ms/step - loss: 0.8297 - accuracy: 0.6976 - val_loss: 0.7827 - val_accuracy: 0.6986
Epoch 7/20
112/112 [==============================] - 32s 284ms/step - loss: 0.7959 - accuracy: 0.7034 - val_loss: 0.8004 - val

In [116]:
new_model_dg_dir  = "D:/models/aug_22/"+dataset+"/C1/"+dataset+"_model_c1_aug_dg_e1"

i=0

for model in models_dg:
    model.save(new_model_dg_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_dg_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_dg_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_dg_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_dg_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_dg_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_dg_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_dg_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_dg_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model

In [ ]:
loading = False

models_dg = []

if loading:
    for i in range(5):
        model_dg_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_gn_e1_"+str(i)
        print(model_dg_dir)
        model =utils.My_model('gtsrb',True,model_dg_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_dg.append(model)
       
    

In [117]:
try:
    del deep_gini_values
    del top_images_by_dg
    del top_labels_by_dg
    del image_sets_dg
    del label_sets_dg
    del models_dg
except:
    print("Error")

In [118]:

gc.collect()

120950

### Softmax 

In [119]:
#softmax values
se_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/softmax_values.npy"
se_values = np.load(se_direction)


In [120]:
# Obtaining top n images by LSA values
top_images_by_se  = utils.get_x_of_indexes(list(np.flip(np.argsort(se_values))),x_train_and_adversary)
top_labels_by_se = utils.get_x_of_indexes(list(np.flip(np.argsort(se_values))),y_train_and_adversary)

In [121]:
m = n_data_points

n = 0
image_sets_se = []
label_sets_se = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_se)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_se)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_se)%m))
        top_images_by_se_n = np.array(top_images_by_se[:n+m+(len(top_images_by_se)%m)])
        top_labels_by_se_n = np.array(top_labels_by_se[:n+m+(len(top_images_by_se)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_se_n = np.array(top_images_by_se[:n+m])
        top_labels_by_se_n = np.array(top_labels_by_se[:n+m])
    image_sets_se.append(top_images_by_se_n)
    label_sets_se.append(top_labels_by_se_n)
    print(len(top_images_by_se_n))
    n += m



0 :
0  ->  700
700
1 :
0  ->  1400
1400
2 :
0  ->  2100
2100
3 :
0  ->  2800
2800
4 :
0  ->  3500
3500
5 :
0  ->  4200
4200
6 :
0  ->  4900
4900
7 :
0  ->  5600
5600
8 :
0  ->  6300
6300
9 :
0  ->  7000
7000
10 :
0  ->  7700
7700
11 :
0  ->  8400
8400
12 :
0  ->  9100
9100
13 :
0  ->  9800
9800
14 :
0  ->  10500
10500
15 :
0  ->  11200
11200
16 :
0  ->  11900
11900
17 :
0  ->  12600
12600
18 :
0  ->  13300
13300
19 :
0  ->  14000
14000
20 :
Last
0  ->  14924
14224


In [122]:
models_se = []
for i in range(len(label_sets_se)):
    print(i,":")
    model = utils.My_model(dataset,False)
    model.compile_model()
    models_se.append(model)

0 :
Model compiled
1 :
Model compiled
2 :
Model compiled
3 :
Model compiled
4 :
Model compiled
5 :
Model compiled
6 :
Model compiled
7 :
Model compiled
8 :
Model compiled
9 :
Model compiled
10 :
Model compiled
11 :
Model compiled
12 :
Model compiled
13 :
Model compiled
14 :
Model compiled
15 :
Model compiled
16 :
Model compiled
17 :
Model compiled
18 :
Model compiled
19 :
Model compiled
20 :
Model compiled


In [123]:
n=0
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

0
Epoch 1/20
6/6 [==============================] - 6s 629ms/step - loss: 1.7953 - accuracy: 0.1735 - val_loss: 1.7935 - val_accuracy: 0.1619
Epoch 2/20
6/6 [==============================] - 3s 518ms/step - loss: 1.7697 - accuracy: 0.2105 - val_loss: 1.7899 - val_accuracy: 0.1790
Epoch 3/20
6/6 [==============================] - 3s 511ms/step - loss: 1.7638 - accuracy: 0.2409 - val_loss: 1.7873 - val_accuracy: 0.1790
Epoch 4/20
6/6 [==============================] - 3s 508ms/step - loss: 1.7495 - accuracy: 0.2424 - val_loss: 1.7745 - val_accuracy: 0.1862
Epoch 5/20
6/6 [==============================] - 3s 518ms/step - loss: 1.7470 - accuracy: 0.2424 - val_loss: 1.7334 - val_accuracy: 0.2093
Epoch 6/20
6/6 [==============================] - 3s 519ms/step - loss: 1.7448 - accuracy: 0.2491 - val_loss: 1.7040 - val_accuracy: 0.2158
Epoch 7/20
6/6 [==============================] - 3s 513ms/step - loss: 1.7056 - accuracy: 0.2613 - val_loss: 1.6830 - val_accuracy: 0.3249
Epoch 8/20
6/6 [==

In [124]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

1
Epoch 1/20
11/11 [==============================] - 8s 447ms/step - loss: 1.7634 - accuracy: 0.2057 - val_loss: 1.7856 - val_accuracy: 0.1776
Epoch 2/20
11/11 [==============================] - 4s 395ms/step - loss: 1.7533 - accuracy: 0.2267 - val_loss: 1.7772 - val_accuracy: 0.1790
Epoch 3/20
11/11 [==============================] - 4s 396ms/step - loss: 1.7121 - accuracy: 0.2507 - val_loss: 1.7296 - val_accuracy: 0.1790
Epoch 4/20
11/11 [==============================] - 4s 393ms/step - loss: 1.6933 - accuracy: 0.2835 - val_loss: 1.6697 - val_accuracy: 0.2422
Epoch 5/20
11/11 [==============================] - 4s 393ms/step - loss: 1.6890 - accuracy: 0.2622 - val_loss: 1.6546 - val_accuracy: 0.2354
Epoch 6/20
11/11 [==============================] - 4s 396ms/step - loss: 1.7004 - accuracy: 0.2582 - val_loss: 1.6106 - val_accuracy: 0.3666
Epoch 7/20
11/11 [==============================] - 4s 392ms/step - loss: 1.6648 - accuracy: 0.3084 - val_loss: 1.6038 - val_accuracy: 0.3916
Epoc

In [125]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

2
Epoch 1/20
17/17 [==============================] - 9s 368ms/step - loss: 1.7556 - accuracy: 0.2522 - val_loss: 1.8061 - val_accuracy: 0.1790
Epoch 2/20
17/17 [==============================] - 6s 329ms/step - loss: 1.7019 - accuracy: 0.2748 - val_loss: 1.7690 - val_accuracy: 0.1794
Epoch 3/20
17/17 [==============================] - 6s 334ms/step - loss: 1.7175 - accuracy: 0.2617 - val_loss: 1.7572 - val_accuracy: 0.1844
Epoch 4/20
17/17 [==============================] - 6s 334ms/step - loss: 1.6893 - accuracy: 0.2698 - val_loss: 1.6991 - val_accuracy: 0.1801
Epoch 5/20
17/17 [==============================] - 6s 336ms/step - loss: 1.6295 - accuracy: 0.3057 - val_loss: 1.6881 - val_accuracy: 0.2243
Epoch 6/20
17/17 [==============================] - 6s 332ms/step - loss: 1.6301 - accuracy: 0.3057 - val_loss: 1.5915 - val_accuracy: 0.3609
Epoch 7/20
17/17 [==============================] - 6s 332ms/step - loss: 1.5992 - accuracy: 0.3288 - val_loss: 1.5189 - val_accuracy: 0.3894
Epoc

In [126]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

3
Epoch 1/20
22/22 [==============================] - 11s 350ms/step - loss: 1.7483 - accuracy: 0.2523 - val_loss: 1.9188 - val_accuracy: 0.1790
Epoch 2/20
22/22 [==============================] - 7s 319ms/step - loss: 1.6576 - accuracy: 0.2880 - val_loss: 1.7103 - val_accuracy: 0.1965
Epoch 3/20
22/22 [==============================] - 7s 316ms/step - loss: 1.6156 - accuracy: 0.3269 - val_loss: 1.4772 - val_accuracy: 0.3370
Epoch 4/20
22/22 [==============================] - 7s 318ms/step - loss: 1.5944 - accuracy: 0.3251 - val_loss: 1.4638 - val_accuracy: 0.4098
Epoch 5/20
22/22 [==============================] - 7s 317ms/step - loss: 1.5765 - accuracy: 0.3434 - val_loss: 1.4758 - val_accuracy: 0.3738
Epoch 6/20
22/22 [==============================] - 7s 315ms/step - loss: 1.5511 - accuracy: 0.3649 - val_loss: 1.5655 - val_accuracy: 0.3459
Epoch 7/20
22/22 [==============================] - 7s 315ms/step - loss: 1.5350 - accuracy: 0.3605 - val_loss: 1.2785 - val_accuracy: 0.4511
Epo

In [127]:
print(n)#
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

4
Epoch 1/20
28/28 [==============================] - 12s 324ms/step - loss: 1.7233 - accuracy: 0.2652 - val_loss: 1.7393 - val_accuracy: 0.2115
Epoch 2/20
28/28 [==============================] - 9s 318ms/step - loss: 1.6433 - accuracy: 0.3095 - val_loss: 1.6109 - val_accuracy: 0.3367
Epoch 3/20
28/28 [==============================] - 9s 308ms/step - loss: 1.5679 - accuracy: 0.3589 - val_loss: 1.4261 - val_accuracy: 0.3766
Epoch 4/20
28/28 [==============================] - 9s 328ms/step - loss: 1.6017 - accuracy: 0.3375 - val_loss: 1.4510 - val_accuracy: 0.3691
Epoch 5/20
28/28 [==============================] - 9s 315ms/step - loss: 1.5346 - accuracy: 0.3654 - val_loss: 1.5016 - val_accuracy: 0.3673
Epoch 6/20
28/28 [==============================] - 9s 313ms/step - loss: 1.5197 - accuracy: 0.3712 - val_loss: 1.3091 - val_accuracy: 0.4244
Epoch 7/20
28/28 [==============================] - 9s 312ms/step - loss: 1.5068 - accuracy: 0.3649 - val_loss: 1.4698 - val_accuracy: 0.3923
Epo

In [128]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

5
Epoch 1/20
33/33 [==============================] - 15s 374ms/step - loss: 1.7948 - accuracy: 0.2571 - val_loss: 1.7732 - val_accuracy: 0.1808
Epoch 2/20
33/33 [==============================] - 11s 335ms/step - loss: 1.6469 - accuracy: 0.3001 - val_loss: 1.6250 - val_accuracy: 0.3228
Epoch 3/20
33/33 [==============================] - 11s 323ms/step - loss: 1.5982 - accuracy: 0.3383 - val_loss: 1.5522 - val_accuracy: 0.3313
Epoch 4/20
33/33 [==============================] - 11s 326ms/step - loss: 1.5752 - accuracy: 0.3473 - val_loss: 1.4913 - val_accuracy: 0.3573
Epoch 5/20
33/33 [==============================] - 12s 352ms/step - loss: 1.5427 - accuracy: 0.3604 - val_loss: 1.3869 - val_accuracy: 0.3738
Epoch 6/20
33/33 [==============================] - 11s 333ms/step - loss: 1.4874 - accuracy: 0.3728 - val_loss: 1.4673 - val_accuracy: 0.4358
Epoch 7/20
33/33 [==============================] - 11s 348ms/step - loss: 1.4610 - accuracy: 0.4008 - val_loss: 1.3106 - val_accuracy: 0.45

In [129]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

6
Epoch 1/20
39/39 [==============================] - 15s 324ms/step - loss: 1.7129 - accuracy: 0.2757 - val_loss: 1.8295 - val_accuracy: 0.2090
Epoch 2/20
39/39 [==============================] - 13s 323ms/step - loss: 1.5979 - accuracy: 0.3393 - val_loss: 1.6654 - val_accuracy: 0.2860
Epoch 3/20
39/39 [==============================] - 12s 320ms/step - loss: 1.5481 - accuracy: 0.3489 - val_loss: 1.3825 - val_accuracy: 0.3905
Epoch 4/20
39/39 [==============================] - 12s 321ms/step - loss: 1.4800 - accuracy: 0.3808 - val_loss: 1.2857 - val_accuracy: 0.4875
Epoch 5/20
39/39 [==============================] - 12s 320ms/step - loss: 1.4438 - accuracy: 0.4052 - val_loss: 1.2639 - val_accuracy: 0.4626
Epoch 6/20
39/39 [==============================] - 13s 323ms/step - loss: 1.4214 - accuracy: 0.4027 - val_loss: 1.4137 - val_accuracy: 0.3916
Epoch 7/20
39/39 [==============================] - 12s 322ms/step - loss: 1.4257 - accuracy: 0.4072 - val_loss: 1.1888 - val_accuracy: 0.52

In [130]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

7
Epoch 1/20
44/44 [==============================] - 17s 320ms/step - loss: 1.7276 - accuracy: 0.2458 - val_loss: 1.6520 - val_accuracy: 0.3317
Epoch 2/20
44/44 [==============================] - 14s 322ms/step - loss: 1.5646 - accuracy: 0.3533 - val_loss: 1.5618 - val_accuracy: 0.3409
Epoch 3/20
44/44 [==============================] - 14s 321ms/step - loss: 1.5041 - accuracy: 0.3843 - val_loss: 1.3055 - val_accuracy: 0.3841
Epoch 4/20
44/44 [==============================] - 14s 321ms/step - loss: 1.4304 - accuracy: 0.3939 - val_loss: 1.3071 - val_accuracy: 0.4290
Epoch 5/20
44/44 [==============================] - 14s 320ms/step - loss: 1.4064 - accuracy: 0.4044 - val_loss: 1.3653 - val_accuracy: 0.4190
Epoch 6/20
44/44 [==============================] - 14s 321ms/step - loss: 1.3591 - accuracy: 0.4256 - val_loss: 1.2821 - val_accuracy: 0.4565
Epoch 7/20
44/44 [==============================] - 14s 323ms/step - loss: 1.3322 - accuracy: 0.4207 - val_loss: 1.2302 - val_accuracy: 0.46

In [131]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

8
Epoch 1/20
50/50 [==============================] - 19s 313ms/step - loss: 1.7237 - accuracy: 0.2576 - val_loss: 1.7470 - val_accuracy: 0.2443
Epoch 2/20
50/50 [==============================] - 16s 314ms/step - loss: 1.5657 - accuracy: 0.3529 - val_loss: 1.4651 - val_accuracy: 0.3884
Epoch 3/20
50/50 [==============================] - 16s 314ms/step - loss: 1.4872 - accuracy: 0.3959 - val_loss: 1.3852 - val_accuracy: 0.3973
Epoch 4/20
50/50 [==============================] - 16s 314ms/step - loss: 1.4330 - accuracy: 0.4159 - val_loss: 1.4054 - val_accuracy: 0.3991
Epoch 5/20
50/50 [==============================] - 16s 317ms/step - loss: 1.4032 - accuracy: 0.4191 - val_loss: 1.1822 - val_accuracy: 0.5061
Epoch 6/20
50/50 [==============================] - 16s 324ms/step - loss: 1.3501 - accuracy: 0.4418 - val_loss: 1.2173 - val_accuracy: 0.4650
Epoch 7/20
50/50 [==============================] - 16s 312ms/step - loss: 1.2883 - accuracy: 0.4583 - val_loss: 1.2195 - val_accuracy: 0.50

In [132]:
print(n)#
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

9
Epoch 1/20
55/55 [==============================] - 23s 339ms/step - loss: 1.7622 - accuracy: 0.2476 - val_loss: 1.6960 - val_accuracy: 0.2746
Epoch 2/20
55/55 [==============================] - 19s 340ms/step - loss: 1.5718 - accuracy: 0.3630 - val_loss: 1.4158 - val_accuracy: 0.3984
Epoch 3/20
55/55 [==============================] - 19s 340ms/step - loss: 1.4650 - accuracy: 0.4030 - val_loss: 1.2624 - val_accuracy: 0.4326
Epoch 4/20
55/55 [==============================] - 19s 340ms/step - loss: 1.3974 - accuracy: 0.4292 - val_loss: 1.2652 - val_accuracy: 0.4511
Epoch 5/20
55/55 [==============================] - 19s 343ms/step - loss: 1.3388 - accuracy: 0.4376 - val_loss: 1.1989 - val_accuracy: 0.4989
Epoch 6/20
55/55 [==============================] - 19s 340ms/step - loss: 1.3034 - accuracy: 0.4484 - val_loss: 1.0980 - val_accuracy: 0.5481
Epoch 7/20
55/55 [==============================] - 19s 339ms/step - loss: 1.2719 - accuracy: 0.4693 - val_loss: 1.1688 - val_accuracy: 0.45

In [133]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

10
Epoch 1/20
61/61 [==============================] - 20s 274ms/step - loss: 1.6878 - accuracy: 0.2662 - val_loss: 1.5883 - val_accuracy: 0.3324
Epoch 2/20
61/61 [==============================] - 16s 269ms/step - loss: 1.4962 - accuracy: 0.3807 - val_loss: 1.4871 - val_accuracy: 0.3741
Epoch 3/20
61/61 [==============================] - 16s 270ms/step - loss: 1.4266 - accuracy: 0.4075 - val_loss: 1.3048 - val_accuracy: 0.4900
Epoch 4/20
61/61 [==============================] - 16s 270ms/step - loss: 1.3724 - accuracy: 0.4305 - val_loss: 1.2213 - val_accuracy: 0.4497
Epoch 5/20
61/61 [==============================] - 17s 274ms/step - loss: 1.3156 - accuracy: 0.4554 - val_loss: 1.2232 - val_accuracy: 0.4939
Epoch 6/20
61/61 [==============================] - 17s 274ms/step - loss: 1.2692 - accuracy: 0.4741 - val_loss: 1.0750 - val_accuracy: 0.5767
Epoch 7/20
61/61 [==============================] - 16s 269ms/step - loss: 1.2472 - accuracy: 0.4859 - val_loss: 1.2977 - val_accuracy: 0.4

In [134]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

11
Epoch 1/20
66/66 [==============================] - 21s 277ms/step - loss: 1.7057 - accuracy: 0.2747 - val_loss: 1.5842 - val_accuracy: 0.3627
Epoch 2/20
66/66 [==============================] - 18s 275ms/step - loss: 1.4859 - accuracy: 0.4018 - val_loss: 1.3142 - val_accuracy: 0.4183
Epoch 3/20
66/66 [==============================] - 18s 280ms/step - loss: 1.4079 - accuracy: 0.4175 - val_loss: 1.2515 - val_accuracy: 0.4486
Epoch 4/20
66/66 [==============================] - 18s 279ms/step - loss: 1.3304 - accuracy: 0.4513 - val_loss: 1.4109 - val_accuracy: 0.4094
Epoch 5/20
66/66 [==============================] - 18s 280ms/step - loss: 1.2877 - accuracy: 0.4683 - val_loss: 1.0690 - val_accuracy: 0.5456
Epoch 6/20
66/66 [==============================] - 22s 332ms/step - loss: 1.2261 - accuracy: 0.4918 - val_loss: 1.2860 - val_accuracy: 0.4918
Epoch 7/20
66/66 [==============================] - 20s 303ms/step - loss: 1.1999 - accuracy: 0.5091 - val_loss: 1.1084 - val_accuracy: 0.5

In [135]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

12
Epoch 1/20
72/72 [==============================] - 25s 308ms/step - loss: 1.6948 - accuracy: 0.2664 - val_loss: 1.5132 - val_accuracy: 0.3966
Epoch 2/20
72/72 [==============================] - 22s 302ms/step - loss: 1.4831 - accuracy: 0.3944 - val_loss: 1.2273 - val_accuracy: 0.4433
Epoch 3/20
72/72 [==============================] - 22s 302ms/step - loss: 1.3967 - accuracy: 0.4484 - val_loss: 1.2152 - val_accuracy: 0.4904
Epoch 4/20
72/72 [==============================] - 22s 304ms/step - loss: 1.2996 - accuracy: 0.4667 - val_loss: 1.5498 - val_accuracy: 0.3816
Epoch 5/20
72/72 [==============================] - 22s 301ms/step - loss: 1.2396 - accuracy: 0.4935 - val_loss: 1.0576 - val_accuracy: 0.5913
Epoch 6/20
72/72 [==============================] - 22s 302ms/step - loss: 1.1842 - accuracy: 0.5211 - val_loss: 1.1339 - val_accuracy: 0.5203
Epoch 7/20
72/72 [==============================] - 22s 302ms/step - loss: 1.1677 - accuracy: 0.5404 - val_loss: 1.1689 - val_accuracy: 0.4

In [136]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

13
Epoch 1/20
77/77 [==============================] - 26s 302ms/step - loss: 1.6792 - accuracy: 0.2628 - val_loss: 1.3859 - val_accuracy: 0.4105
Epoch 2/20
77/77 [==============================] - 23s 301ms/step - loss: 1.4073 - accuracy: 0.4167 - val_loss: 1.1538 - val_accuracy: 0.4786
Epoch 3/20
77/77 [==============================] - 23s 302ms/step - loss: 1.3064 - accuracy: 0.4578 - val_loss: 1.1123 - val_accuracy: 0.5082
Epoch 4/20
77/77 [==============================] - 23s 302ms/step - loss: 1.2182 - accuracy: 0.5024 - val_loss: 1.0790 - val_accuracy: 0.5200
Epoch 5/20
77/77 [==============================] - 23s 302ms/step - loss: 1.1702 - accuracy: 0.5299 - val_loss: 1.1478 - val_accuracy: 0.5335
Epoch 6/20
77/77 [==============================] - 23s 301ms/step - loss: 1.1032 - accuracy: 0.5563 - val_loss: 1.0096 - val_accuracy: 0.5934
Epoch 7/20
77/77 [==============================] - 23s 302ms/step - loss: 1.0641 - accuracy: 0.5753 - val_loss: 0.8795 - val_accuracy: 0.6

In [137]:
print(n)#
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

14
Epoch 1/20
83/83 [==============================] - 31s 323ms/step - loss: 1.6521 - accuracy: 0.2785 - val_loss: 1.6066 - val_accuracy: 0.3495
Epoch 2/20
83/83 [==============================] - 27s 325ms/step - loss: 1.4021 - accuracy: 0.4307 - val_loss: 1.4796 - val_accuracy: 0.3680
Epoch 3/20
83/83 [==============================] - 27s 326ms/step - loss: 1.3125 - accuracy: 0.4708 - val_loss: 1.4208 - val_accuracy: 0.4636
Epoch 4/20
83/83 [==============================] - 27s 326ms/step - loss: 1.2020 - accuracy: 0.5182 - val_loss: 1.0542 - val_accuracy: 0.5631
Epoch 5/20
83/83 [==============================] - 28s 332ms/step - loss: 1.1571 - accuracy: 0.5386 - val_loss: 0.9925 - val_accuracy: 0.6266
Epoch 6/20
83/83 [==============================] - 27s 328ms/step - loss: 1.0976 - accuracy: 0.5540 - val_loss: 1.3169 - val_accuracy: 0.4294
Epoch 7/20
83/83 [==============================] - 29s 348ms/step - loss: 1.0623 - accuracy: 0.5853 - val_loss: 1.1802 - val_accuracy: 0.5

In [138]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

15
Epoch 1/20
88/88 [==============================] - 33s 334ms/step - loss: 1.6819 - accuracy: 0.2641 - val_loss: 1.4943 - val_accuracy: 0.3613
Epoch 2/20
88/88 [==============================] - 29s 328ms/step - loss: 1.3665 - accuracy: 0.4408 - val_loss: 1.1779 - val_accuracy: 0.5153
Epoch 3/20
88/88 [==============================] - 31s 347ms/step - loss: 1.2854 - accuracy: 0.4729 - val_loss: 1.3016 - val_accuracy: 0.4604
Epoch 4/20
88/88 [==============================] - 29s 328ms/step - loss: 1.1736 - accuracy: 0.5314 - val_loss: 1.0016 - val_accuracy: 0.6073
Epoch 5/20
88/88 [==============================] - 29s 326ms/step - loss: 1.1040 - accuracy: 0.5536 - val_loss: 1.0366 - val_accuracy: 0.5899
Epoch 6/20
88/88 [==============================] - 29s 328ms/step - loss: 1.0627 - accuracy: 0.5818 - val_loss: 0.8113 - val_accuracy: 0.7040
Epoch 7/20
88/88 [==============================] - 29s 326ms/step - loss: 0.9800 - accuracy: 0.6222 - val_loss: 0.8279 - val_accuracy: 0.7

In [139]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

16
Epoch 1/20
93/93 [==============================] - 35s 327ms/step - loss: 1.7119 - accuracy: 0.2580 - val_loss: 1.3940 - val_accuracy: 0.4083
Epoch 2/20
93/93 [==============================] - 31s 330ms/step - loss: 1.4059 - accuracy: 0.4282 - val_loss: 1.1136 - val_accuracy: 0.5792
Epoch 3/20
93/93 [==============================] - 30s 327ms/step - loss: 1.2649 - accuracy: 0.4977 - val_loss: 1.2889 - val_accuracy: 0.5185
Epoch 4/20
93/93 [==============================] - 31s 331ms/step - loss: 1.1751 - accuracy: 0.5327 - val_loss: 0.9405 - val_accuracy: 0.6373
Epoch 5/20
93/93 [==============================] - 33s 353ms/step - loss: 1.0797 - accuracy: 0.5798 - val_loss: 0.8856 - val_accuracy: 0.6748
Epoch 6/20
93/93 [==============================] - 32s 340ms/step - loss: 1.0110 - accuracy: 0.6152 - val_loss: 0.8638 - val_accuracy: 0.6737
Epoch 7/20
93/93 [==============================] - 31s 329ms/step - loss: 0.9965 - accuracy: 0.6211 - val_loss: 0.7932 - val_accuracy: 0.6

In [140]:
time.sleep(400)

In [141]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

17
Epoch 1/20
99/99 [==============================] - 29s 265ms/step - loss: 1.6806 - accuracy: 0.2741 - val_loss: 1.2775 - val_accuracy: 0.5096
Epoch 2/20
99/99 [==============================] - 26s 262ms/step - loss: 1.3353 - accuracy: 0.4614 - val_loss: 1.4908 - val_accuracy: 0.4608
Epoch 3/20
99/99 [==============================] - 26s 263ms/step - loss: 1.2113 - accuracy: 0.5245 - val_loss: 0.9222 - val_accuracy: 0.6416
Epoch 4/20
99/99 [==============================] - 26s 263ms/step - loss: 1.0770 - accuracy: 0.5821 - val_loss: 0.9643 - val_accuracy: 0.6287
Epoch 5/20
99/99 [==============================] - 26s 265ms/step - loss: 0.9994 - accuracy: 0.6196 - val_loss: 0.8484 - val_accuracy: 0.7068
Epoch 6/20
99/99 [==============================] - 26s 265ms/step - loss: 0.9504 - accuracy: 0.6362 - val_loss: 0.7831 - val_accuracy: 0.7193
Epoch 7/20
99/99 [==============================] - 26s 264ms/step - loss: 0.9181 - accuracy: 0.6594 - val_loss: 0.7499 - val_accuracy: 0.7

In [142]:
time.sleep(400)

In [143]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

18
Epoch 1/20
104/104 [==============================] - 31s 268ms/step - loss: 1.6961 - accuracy: 0.2671 - val_loss: 1.2430 - val_accuracy: 0.4864
Epoch 2/20
104/104 [==============================] - 27s 264ms/step - loss: 1.3391 - accuracy: 0.4553 - val_loss: 0.9809 - val_accuracy: 0.6120
Epoch 3/20
104/104 [==============================] - 28s 265ms/step - loss: 1.1488 - accuracy: 0.5544 - val_loss: 1.0101 - val_accuracy: 0.6070
Epoch 4/20
104/104 [==============================] - 27s 264ms/step - loss: 1.0373 - accuracy: 0.5983 - val_loss: 0.8455 - val_accuracy: 0.6676
Epoch 5/20
104/104 [==============================] - 27s 264ms/step - loss: 0.9779 - accuracy: 0.6195 - val_loss: 0.7741 - val_accuracy: 0.7101
Epoch 6/20
104/104 [==============================] - 28s 268ms/step - loss: 0.9171 - accuracy: 0.6564 - val_loss: 0.7623 - val_accuracy: 0.7054
Epoch 7/20
104/104 [==============================] - 27s 264ms/step - loss: 0.8765 - accuracy: 0.6711 - val_loss: 0.6904 - val

In [144]:
time.sleep(400)

In [145]:
print(n)#
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

19
Epoch 1/20
110/110 [==============================] - 32s 265ms/step - loss: 1.6999 - accuracy: 0.2630 - val_loss: 1.1908 - val_accuracy: 0.5428
Epoch 2/20
110/110 [==============================] - 29s 261ms/step - loss: 1.2821 - accuracy: 0.4918 - val_loss: 1.0531 - val_accuracy: 0.5717
Epoch 3/20
110/110 [==============================] - 29s 266ms/step - loss: 1.1306 - accuracy: 0.5528 - val_loss: 0.9935 - val_accuracy: 0.6202
Epoch 4/20
110/110 [==============================] - 29s 263ms/step - loss: 1.0274 - accuracy: 0.6017 - val_loss: 0.9115 - val_accuracy: 0.6594
Epoch 5/20
110/110 [==============================] - 29s 264ms/step - loss: 0.9388 - accuracy: 0.6379 - val_loss: 1.1429 - val_accuracy: 0.5328
Epoch 6/20
110/110 [==============================] - 29s 264ms/step - loss: 0.8989 - accuracy: 0.6635 - val_loss: 0.8812 - val_accuracy: 0.6815
Epoch 7/20
110/110 [==============================] - 29s 266ms/step - loss: 0.8354 - accuracy: 0.6842 - val_loss: 0.7266 - val

In [ ]:
time.sleep(400)

In [146]:
print(n)#
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

20
Epoch 1/20
112/112 [==============================] - 36s 292ms/step - loss: 1.6536 - accuracy: 0.2871 - val_loss: 1.2250 - val_accuracy: 0.5460
Epoch 2/20
112/112 [==============================] - 33s 294ms/step - loss: 1.2475 - accuracy: 0.5005 - val_loss: 1.4006 - val_accuracy: 0.4419
Epoch 3/20
112/112 [==============================] - 33s 295ms/step - loss: 1.1093 - accuracy: 0.5718 - val_loss: 0.9489 - val_accuracy: 0.6180
Epoch 4/20
112/112 [==============================] - 33s 298ms/step - loss: 0.9577 - accuracy: 0.6266 - val_loss: 1.3625 - val_accuracy: 0.4979
Epoch 5/20
112/112 [==============================] - 34s 303ms/step - loss: 0.9318 - accuracy: 0.6526 - val_loss: 1.1244 - val_accuracy: 0.5788
Epoch 6/20
112/112 [==============================] - 36s 322ms/step - loss: 0.8693 - accuracy: 0.6785 - val_loss: 0.7718 - val_accuracy: 0.6979
Epoch 7/20
112/112 [==============================] - 38s 341ms/step - loss: 0.8067 - accuracy: 0.7026 - val_loss: 0.6648 - val

In [ ]:
len(models_se)

In [147]:
new_model_se_dir  = "D:/models/aug_22/"+dataset+"/C1/"+dataset+"_model_c1_aug_se_e1"

i=0

for model in models_se:
    model.save(new_model_se_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_se_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_se_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_se_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_se_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_se_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_se_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_se_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_se_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model

In [148]:
try:
    del se_values
    del top_images_by_se
    del top_labels_by_se
    del image_sets_se
    del label_sets_se
    del models_se
except:
    print("Error")

In [149]:
gc.collect()

120950

## Training guided by Random values

In [ ]:
import random
random_indexes =list(range(len(x_train_and_adversary)))
random.shuffle(random_indexes)
print(random_indexes[:10])
print(len(random_indexes))

In [150]:
save_dir = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/random_values.npy"


random_indexes = np.load(save_dir)

In [151]:
len(random_indexes)

14224

In [152]:
# Obtaining top n images by random values
top_images_by_random = utils.get_x_of_indexes(list(np.flip(np.argsort(random_indexes))),x_train_and_adversary)
top_labels_by_random = utils.get_x_of_indexes(list(np.flip(np.argsort(random_indexes))),y_train_and_adversary)

In [153]:
m = n_data_points
n = 0
image_sets_random = []
label_sets_random = []



for i in range((len(top_images_by_random)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_random)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_random)%m))
        top_images_by_random_n = np.array(top_images_by_random[:n+m+(len(top_images_by_random)%m)])
        top_labels_by_random_n = np.array(top_labels_by_random[:n+m+(len(top_images_by_random)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_random_n = np.array(top_images_by_random[:n+m])
        top_labels_by_random_n = np.array(top_labels_by_random[:n+m])
    image_sets_random.append(top_images_by_random_n)
    label_sets_random.append(top_labels_by_random_n)
    print(len(top_images_by_random_n))
    n += m




0 :
0  ->  700
700
1 :
0  ->  1400
1400
2 :
0  ->  2100
2100
3 :
0  ->  2800
2800
4 :
0  ->  3500
3500
5 :
0  ->  4200
4200
6 :
0  ->  4900
4900
7 :
0  ->  5600
5600
8 :
0  ->  6300
6300
9 :
0  ->  7000
7000
10 :
0  ->  7700
7700
11 :
0  ->  8400
8400
12 :
0  ->  9100
9100
13 :
0  ->  9800
9800
14 :
0  ->  10500
10500
15 :
0  ->  11200
11200
16 :
0  ->  11900
11900
17 :
0  ->  12600
12600
18 :
0  ->  13300
13300
19 :
0  ->  14000
14000
20 :
Last
0  ->  14924
14224


In [154]:
len(image_sets_random)

21

In [155]:
print(model_dir)

models_random = []
for i in range(len(label_sets_random)):
    print(i,":")
    model = utils.My_model(dataset,False,None)
    model.compile_model()
    models_random.append(model)


C:/Users/fjdur/Documents/upc-july/models/intel_model_21_10
0 :
Model compiled
1 :
Model compiled
2 :
Model compiled
3 :
Model compiled
4 :
Model compiled
5 :
Model compiled
6 :
Model compiled
7 :
Model compiled
8 :
Model compiled
9 :
Model compiled
10 :
Model compiled
11 :
Model compiled
12 :
Model compiled
13 :
Model compiled
14 :
Model compiled
15 :
Model compiled
16 :
Model compiled
17 :
Model compiled
18 :
Model compiled
19 :
Model compiled
20 :
Model compiled


In [156]:
n=0
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

0
Epoch 1/20
6/6 [==============================] - 8s 714ms/step - loss: 1.8730 - accuracy: 0.1878 - val_loss: 1.7886 - val_accuracy: 0.1844
Epoch 2/20
6/6 [==============================] - 3s 548ms/step - loss: 1.7885 - accuracy: 0.2047 - val_loss: 1.7841 - val_accuracy: 0.2019
Epoch 3/20
6/6 [==============================] - 3s 559ms/step - loss: 1.7715 - accuracy: 0.2145 - val_loss: 1.8037 - val_accuracy: 0.1698
Epoch 4/20
6/6 [==============================] - 3s 544ms/step - loss: 1.7928 - accuracy: 0.2145 - val_loss: 1.6804 - val_accuracy: 0.4280
Epoch 5/20
6/6 [==============================] - 3s 527ms/step - loss: 1.6983 - accuracy: 0.2972 - val_loss: 1.6051 - val_accuracy: 0.3014
Epoch 6/20
6/6 [==============================] - 3s 531ms/step - loss: 2.1027 - accuracy: 0.2509 - val_loss: 1.5058 - val_accuracy: 0.4265
Epoch 7/20
6/6 [==============================] - 3s 523ms/step - loss: 1.5393 - accuracy: 0.4193 - val_loss: 1.4505 - val_accuracy: 0.4219
Epoch 8/20
6/6 [==

In [157]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

1
Epoch 1/20
11/11 [==============================] - 8s 451ms/step - loss: 1.8042 - accuracy: 0.1642 - val_loss: 1.7855 - val_accuracy: 0.1701
Epoch 2/20
11/11 [==============================] - 4s 399ms/step - loss: 1.7851 - accuracy: 0.1689 - val_loss: 1.7360 - val_accuracy: 0.2104
Epoch 3/20
11/11 [==============================] - 4s 401ms/step - loss: 1.7860 - accuracy: 0.2493 - val_loss: 1.5375 - val_accuracy: 0.4276
Epoch 4/20
11/11 [==============================] - 4s 404ms/step - loss: 1.6654 - accuracy: 0.2878 - val_loss: 1.5205 - val_accuracy: 0.3837
Epoch 5/20
11/11 [==============================] - 4s 402ms/step - loss: 1.5629 - accuracy: 0.3569 - val_loss: 1.4119 - val_accuracy: 0.4397
Epoch 6/20
11/11 [==============================] - 4s 399ms/step - loss: 1.5061 - accuracy: 0.3951 - val_loss: 1.3599 - val_accuracy: 0.4968
Epoch 7/20
11/11 [==============================] - 4s 406ms/step - loss: 1.4440 - accuracy: 0.4038 - val_loss: 1.4936 - val_accuracy: 0.4091
Epoc

In [158]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

2
Epoch 1/20
17/17 [==============================] - 9s 371ms/step - loss: 1.8388 - accuracy: 0.1860 - val_loss: 1.7943 - val_accuracy: 0.1830
Epoch 2/20
17/17 [==============================] - 6s 336ms/step - loss: 1.7962 - accuracy: 0.2019 - val_loss: 1.6519 - val_accuracy: 0.3249
Epoch 3/20
17/17 [==============================] - 6s 338ms/step - loss: 1.6271 - accuracy: 0.3496 - val_loss: 1.5559 - val_accuracy: 0.4094
Epoch 4/20
17/17 [==============================] - 6s 341ms/step - loss: 1.5774 - accuracy: 0.3534 - val_loss: 1.3694 - val_accuracy: 0.4882
Epoch 5/20
17/17 [==============================] - 6s 338ms/step - loss: 1.4179 - accuracy: 0.4350 - val_loss: 1.2430 - val_accuracy: 0.5364
Epoch 6/20
17/17 [==============================] - 6s 337ms/step - loss: 1.3469 - accuracy: 0.4557 - val_loss: 1.1794 - val_accuracy: 0.5189
Epoch 7/20
17/17 [==============================] - 6s 338ms/step - loss: 1.3820 - accuracy: 0.4617 - val_loss: 1.1436 - val_accuracy: 0.5389
Epoc

In [159]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

3
Epoch 1/20
22/22 [==============================] - 11s 369ms/step - loss: 1.7880 - accuracy: 0.1867 - val_loss: 1.7013 - val_accuracy: 0.3409
Epoch 2/20
22/22 [==============================] - 7s 338ms/step - loss: 1.5936 - accuracy: 0.3500 - val_loss: 1.4320 - val_accuracy: 0.4212
Epoch 3/20
22/22 [==============================] - 7s 328ms/step - loss: 1.5184 - accuracy: 0.3749 - val_loss: 1.3581 - val_accuracy: 0.4205
Epoch 4/20
22/22 [==============================] - 7s 322ms/step - loss: 1.4288 - accuracy: 0.4263 - val_loss: 1.1727 - val_accuracy: 0.5439
Epoch 5/20
22/22 [==============================] - 7s 323ms/step - loss: 1.3317 - accuracy: 0.4673 - val_loss: 1.2434 - val_accuracy: 0.5064
Epoch 6/20
22/22 [==============================] - 7s 322ms/step - loss: 1.2876 - accuracy: 0.4803 - val_loss: 1.1059 - val_accuracy: 0.5656
Epoch 7/20
22/22 [==============================] - 7s 323ms/step - loss: 1.1989 - accuracy: 0.5201 - val_loss: 1.1166 - val_accuracy: 0.5553
Epo

In [160]:
print(n)#

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

4
Epoch 1/20
28/28 [==============================] - 12s 319ms/step - loss: 1.7993 - accuracy: 0.1892 - val_loss: 1.7678 - val_accuracy: 0.2628
Epoch 2/20
28/28 [==============================] - 8s 304ms/step - loss: 1.6231 - accuracy: 0.3270 - val_loss: 1.3367 - val_accuracy: 0.4626
Epoch 3/20
28/28 [==============================] - 9s 316ms/step - loss: 1.4560 - accuracy: 0.4072 - val_loss: 1.2993 - val_accuracy: 0.4790
Epoch 4/20
28/28 [==============================] - 9s 319ms/step - loss: 1.3251 - accuracy: 0.4693 - val_loss: 1.5676 - val_accuracy: 0.3966
Epoch 5/20
28/28 [==============================] - 9s 321ms/step - loss: 1.3071 - accuracy: 0.5018 - val_loss: 1.1149 - val_accuracy: 0.5464
Epoch 6/20
28/28 [==============================] - 9s 317ms/step - loss: 1.2429 - accuracy: 0.5020 - val_loss: 1.1442 - val_accuracy: 0.5752
Epoch 7/20
28/28 [==============================] - 9s 316ms/step - loss: 1.1823 - accuracy: 0.5368 - val_loss: 1.1634 - val_accuracy: 0.5392
Epo

In [161]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

5
Epoch 1/20
33/33 [==============================] - 14s 331ms/step - loss: 1.7976 - accuracy: 0.1881 - val_loss: 1.6925 - val_accuracy: 0.2999
Epoch 2/20
33/33 [==============================] - 11s 338ms/step - loss: 1.5997 - accuracy: 0.3314 - val_loss: 1.3646 - val_accuracy: 0.4989
Epoch 3/20
33/33 [==============================] - 11s 338ms/step - loss: 1.4211 - accuracy: 0.4203 - val_loss: 1.1507 - val_accuracy: 0.5596
Epoch 4/20
33/33 [==============================] - 11s 336ms/step - loss: 1.3184 - accuracy: 0.4732 - val_loss: 1.1006 - val_accuracy: 0.5613
Epoch 5/20
33/33 [==============================] - 11s 340ms/step - loss: 1.2562 - accuracy: 0.4796 - val_loss: 1.0163 - val_accuracy: 0.5959
Epoch 6/20
33/33 [==============================] - 11s 340ms/step - loss: 1.2154 - accuracy: 0.5275 - val_loss: 1.1255 - val_accuracy: 0.5350
Epoch 7/20
33/33 [==============================] - 11s 340ms/step - loss: 1.1777 - accuracy: 0.5271 - val_loss: 1.3712 - val_accuracy: 0.49

In [162]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

6
Epoch 1/20
39/39 [==============================] - 16s 329ms/step - loss: 1.7613 - accuracy: 0.2056 - val_loss: 1.3322 - val_accuracy: 0.3941
Epoch 2/20
39/39 [==============================] - 13s 334ms/step - loss: 1.4299 - accuracy: 0.3974 - val_loss: 1.3126 - val_accuracy: 0.4626
Epoch 3/20
39/39 [==============================] - 13s 333ms/step - loss: 1.3001 - accuracy: 0.4519 - val_loss: 1.1706 - val_accuracy: 0.5253
Epoch 4/20
39/39 [==============================] - 13s 333ms/step - loss: 1.2017 - accuracy: 0.5134 - val_loss: 1.1512 - val_accuracy: 0.5064
Epoch 5/20
39/39 [==============================] - 13s 334ms/step - loss: 1.1446 - accuracy: 0.5425 - val_loss: 1.0018 - val_accuracy: 0.5863
Epoch 6/20
39/39 [==============================] - 13s 332ms/step - loss: 1.0744 - accuracy: 0.5713 - val_loss: 0.8968 - val_accuracy: 0.6427
Epoch 7/20
39/39 [==============================] - 13s 337ms/step - loss: 1.0820 - accuracy: 0.5951 - val_loss: 1.0288 - val_accuracy: 0.58

In [163]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

7
Epoch 1/20
44/44 [==============================] - 17s 322ms/step - loss: 1.7726 - accuracy: 0.2247 - val_loss: 1.4249 - val_accuracy: 0.4608
Epoch 2/20
44/44 [==============================] - 14s 322ms/step - loss: 1.4979 - accuracy: 0.4046 - val_loss: 1.2292 - val_accuracy: 0.5228
Epoch 3/20
44/44 [==============================] - 15s 334ms/step - loss: 1.3799 - accuracy: 0.4648 - val_loss: 1.0964 - val_accuracy: 0.5596
Epoch 4/20
44/44 [==============================] - 14s 322ms/step - loss: 1.2068 - accuracy: 0.5233 - val_loss: 1.2594 - val_accuracy: 0.4875
Epoch 5/20
44/44 [==============================] - 14s 323ms/step - loss: 1.1430 - accuracy: 0.5486 - val_loss: 1.1188 - val_accuracy: 0.5542
Epoch 6/20
44/44 [==============================] - 14s 326ms/step - loss: 1.1595 - accuracy: 0.5438 - val_loss: 0.9661 - val_accuracy: 0.6312
Epoch 7/20
44/44 [==============================] - 14s 327ms/step - loss: 1.0491 - accuracy: 0.5965 - val_loss: 0.9594 - val_accuracy: 0.64

In [164]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

8
Epoch 1/20
50/50 [==============================] - 19s 318ms/step - loss: 1.7795 - accuracy: 0.1942 - val_loss: 1.5776 - val_accuracy: 0.3049
Epoch 2/20
50/50 [==============================] - 16s 318ms/step - loss: 1.4533 - accuracy: 0.4031 - val_loss: 1.2540 - val_accuracy: 0.5004
Epoch 3/20
50/50 [==============================] - 16s 319ms/step - loss: 1.2779 - accuracy: 0.4878 - val_loss: 1.3934 - val_accuracy: 0.4668
Epoch 4/20
50/50 [==============================] - 16s 314ms/step - loss: 1.2114 - accuracy: 0.5116 - val_loss: 1.1802 - val_accuracy: 0.5103
Epoch 5/20
50/50 [==============================] - 16s 315ms/step - loss: 1.1344 - accuracy: 0.5596 - val_loss: 1.0625 - val_accuracy: 0.6066
Epoch 6/20
50/50 [==============================] - 16s 314ms/step - loss: 1.0573 - accuracy: 0.5780 - val_loss: 1.2594 - val_accuracy: 0.5011
Epoch 7/20
50/50 [==============================] - 16s 314ms/step - loss: 1.0450 - accuracy: 0.5927 - val_loss: 0.8637 - val_accuracy: 0.66

In [165]:
print(n)#

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

9
Epoch 1/20
55/55 [==============================] - 23s 347ms/step - loss: 1.7727 - accuracy: 0.2114 - val_loss: 1.3116 - val_accuracy: 0.4636
Epoch 2/20
55/55 [==============================] - 19s 345ms/step - loss: 1.4255 - accuracy: 0.4293 - val_loss: 1.3268 - val_accuracy: 0.4864
Epoch 3/20
55/55 [==============================] - 19s 348ms/step - loss: 1.2320 - accuracy: 0.5162 - val_loss: 1.0071 - val_accuracy: 0.5995
Epoch 4/20
55/55 [==============================] - 19s 345ms/step - loss: 1.1556 - accuracy: 0.5524 - val_loss: 0.9898 - val_accuracy: 0.6155
Epoch 5/20
55/55 [==============================] - 19s 344ms/step - loss: 1.0793 - accuracy: 0.5782 - val_loss: 0.9489 - val_accuracy: 0.6348
Epoch 6/20
55/55 [==============================] - 19s 344ms/step - loss: 1.0083 - accuracy: 0.6049 - val_loss: 0.8988 - val_accuracy: 0.6462
Epoch 7/20
55/55 [==============================] - 19s 346ms/step - loss: 0.9835 - accuracy: 0.6206 - val_loss: 0.8302 - val_accuracy: 0.68

In [166]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

10
Epoch 1/20
61/61 [==============================] - 26s 343ms/step - loss: 1.7580 - accuracy: 0.2285 - val_loss: 1.6994 - val_accuracy: 0.2907
Epoch 2/20
61/61 [==============================] - 21s 340ms/step - loss: 1.4627 - accuracy: 0.4017 - val_loss: 1.2110 - val_accuracy: 0.5289
Epoch 3/20
61/61 [==============================] - 21s 338ms/step - loss: 1.2724 - accuracy: 0.5086 - val_loss: 1.1802 - val_accuracy: 0.5481
Epoch 4/20
61/61 [==============================] - 21s 337ms/step - loss: 1.1333 - accuracy: 0.5503 - val_loss: 1.0529 - val_accuracy: 0.5806
Epoch 5/20
61/61 [==============================] - 21s 338ms/step - loss: 1.1046 - accuracy: 0.5593 - val_loss: 0.9402 - val_accuracy: 0.6302
Epoch 6/20
61/61 [==============================] - 21s 338ms/step - loss: 1.0010 - accuracy: 0.6172 - val_loss: 1.2595 - val_accuracy: 0.5029
Epoch 7/20
61/61 [==============================] - 21s 338ms/step - loss: 1.0104 - accuracy: 0.6038 - val_loss: 1.1101 - val_accuracy: 0.5

In [167]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

11
Epoch 1/20
66/66 [==============================] - 26s 335ms/step - loss: 1.7618 - accuracy: 0.2153 - val_loss: 1.2899 - val_accuracy: 0.4793
Epoch 2/20
66/66 [==============================] - 22s 338ms/step - loss: 1.3880 - accuracy: 0.4366 - val_loss: 1.2670 - val_accuracy: 0.4954
Epoch 3/20
66/66 [==============================] - 22s 338ms/step - loss: 1.2497 - accuracy: 0.5018 - val_loss: 1.0117 - val_accuracy: 0.6109
Epoch 4/20
66/66 [==============================] - 22s 339ms/step - loss: 1.1321 - accuracy: 0.5472 - val_loss: 1.0045 - val_accuracy: 0.5802
Epoch 5/20
66/66 [==============================] - 22s 340ms/step - loss: 1.0670 - accuracy: 0.5768 - val_loss: 0.9436 - val_accuracy: 0.6284
Epoch 6/20
66/66 [==============================] - 24s 360ms/step - loss: 0.9943 - accuracy: 0.6180 - val_loss: 1.0119 - val_accuracy: 0.6077
Epoch 7/20
66/66 [==============================] - 23s 343ms/step - loss: 0.9655 - accuracy: 0.6291 - val_loss: 0.8274 - val_accuracy: 0.6

In [168]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

12
Epoch 1/20
72/72 [==============================] - 28s 332ms/step - loss: 1.7626 - accuracy: 0.2301 - val_loss: 1.4127 - val_accuracy: 0.4322
Epoch 2/20
72/72 [==============================] - 24s 338ms/step - loss: 1.4013 - accuracy: 0.4369 - val_loss: 1.3121 - val_accuracy: 0.5018
Epoch 3/20
72/72 [==============================] - 25s 344ms/step - loss: 1.2156 - accuracy: 0.5322 - val_loss: 1.2956 - val_accuracy: 0.4708
Epoch 4/20
72/72 [==============================] - 24s 334ms/step - loss: 1.1196 - accuracy: 0.5609 - val_loss: 1.1207 - val_accuracy: 0.5599
Epoch 5/20
72/72 [==============================] - 24s 335ms/step - loss: 1.0623 - accuracy: 0.5936 - val_loss: 0.9640 - val_accuracy: 0.6195
Epoch 6/20
72/72 [==============================] - 24s 339ms/step - loss: 0.9942 - accuracy: 0.6176 - val_loss: 0.9323 - val_accuracy: 0.6480
Epoch 7/20
72/72 [==============================] - 24s 333ms/step - loss: 0.9347 - accuracy: 0.6437 - val_loss: 1.3685 - val_accuracy: 0.4

In [169]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

13
Epoch 1/20
77/77 [==============================] - 30s 335ms/step - loss: 1.7557 - accuracy: 0.2258 - val_loss: 1.3481 - val_accuracy: 0.4740
Epoch 2/20
77/77 [==============================] - 26s 336ms/step - loss: 1.3784 - accuracy: 0.4436 - val_loss: 1.0817 - val_accuracy: 0.5749
Epoch 3/20
77/77 [==============================] - 26s 337ms/step - loss: 1.2095 - accuracy: 0.5186 - val_loss: 0.9796 - val_accuracy: 0.5999
Epoch 4/20
77/77 [==============================] - 26s 337ms/step - loss: 1.1119 - accuracy: 0.5710 - val_loss: 0.9686 - val_accuracy: 0.6123
Epoch 5/20
77/77 [==============================] - 26s 338ms/step - loss: 1.0170 - accuracy: 0.6056 - val_loss: 0.8574 - val_accuracy: 0.6708
Epoch 6/20
77/77 [==============================] - 26s 335ms/step - loss: 0.9626 - accuracy: 0.6224 - val_loss: 0.9630 - val_accuracy: 0.6295
Epoch 7/20
77/77 [==============================] - 26s 336ms/step - loss: 0.9131 - accuracy: 0.6467 - val_loss: 0.7920 - val_accuracy: 0.6

In [170]:
print(n)#

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

14
Epoch 1/20
83/83 [==============================] - 32s 332ms/step - loss: 1.7310 - accuracy: 0.2469 - val_loss: 1.3444 - val_accuracy: 0.4529
Epoch 2/20
83/83 [==============================] - 28s 333ms/step - loss: 1.3550 - accuracy: 0.4576 - val_loss: 1.8033 - val_accuracy: 0.4187
Epoch 3/20
83/83 [==============================] - 28s 334ms/step - loss: 1.2702 - accuracy: 0.5184 - val_loss: 1.6013 - val_accuracy: 0.3666
Epoch 4/20
83/83 [==============================] - 28s 332ms/step - loss: 1.1179 - accuracy: 0.5678 - val_loss: 1.5326 - val_accuracy: 0.5086
Epoch 5/20
83/83 [==============================] - 28s 333ms/step - loss: 1.0382 - accuracy: 0.6144 - val_loss: 1.0776 - val_accuracy: 0.5699
Epoch 6/20
83/83 [==============================] - 28s 341ms/step - loss: 0.9641 - accuracy: 0.6303 - val_loss: 0.8509 - val_accuracy: 0.6865
Epoch 7/20
83/83 [==============================] - 29s 345ms/step - loss: 0.9031 - accuracy: 0.6673 - val_loss: 0.8497 - val_accuracy: 0.6

In [171]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

15
Epoch 1/20
88/88 [==============================] - 34s 334ms/step - loss: 1.7010 - accuracy: 0.2531 - val_loss: 1.3656 - val_accuracy: 0.4034
Epoch 2/20
88/88 [==============================] - 29s 334ms/step - loss: 1.3131 - accuracy: 0.4698 - val_loss: 0.9986 - val_accuracy: 0.6170
Epoch 3/20
88/88 [==============================] - 29s 334ms/step - loss: 1.1368 - accuracy: 0.5467 - val_loss: 0.9720 - val_accuracy: 0.6316
Epoch 4/20
88/88 [==============================] - 30s 338ms/step - loss: 1.0439 - accuracy: 0.5897 - val_loss: 0.8863 - val_accuracy: 0.6487
Epoch 5/20
88/88 [==============================] - 29s 334ms/step - loss: 0.9733 - accuracy: 0.6340 - val_loss: 0.8290 - val_accuracy: 0.6812
Epoch 6/20
88/88 [==============================] - 29s 334ms/step - loss: 0.9127 - accuracy: 0.6612 - val_loss: 0.9540 - val_accuracy: 0.6398
Epoch 7/20
88/88 [==============================] - 29s 334ms/step - loss: 0.8563 - accuracy: 0.6824 - val_loss: 0.8551 - val_accuracy: 0.6

In [172]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

16
Epoch 1/20
93/93 [==============================] - 36s 340ms/step - loss: 1.7030 - accuracy: 0.2616 - val_loss: 1.1816 - val_accuracy: 0.5282
Epoch 2/20
93/93 [==============================] - 31s 338ms/step - loss: 1.2902 - accuracy: 0.4800 - val_loss: 1.1518 - val_accuracy: 0.5603
Epoch 3/20
93/93 [==============================] - 31s 336ms/step - loss: 1.1507 - accuracy: 0.5473 - val_loss: 0.9337 - val_accuracy: 0.6430
Epoch 4/20
93/93 [==============================] - 31s 339ms/step - loss: 1.0497 - accuracy: 0.5831 - val_loss: 1.0846 - val_accuracy: 0.5745
Epoch 5/20
93/93 [==============================] - 31s 335ms/step - loss: 0.9763 - accuracy: 0.6267 - val_loss: 0.9914 - val_accuracy: 0.6170
Epoch 6/20
93/93 [==============================] - 31s 335ms/step - loss: 0.9164 - accuracy: 0.6596 - val_loss: 0.9868 - val_accuracy: 0.6345
Epoch 7/20
93/93 [==============================] - 31s 335ms/step - loss: 0.8720 - accuracy: 0.6724 - val_loss: 0.9120 - val_accuracy: 0.6

In [173]:
time.sleep(400)

In [174]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

17
Epoch 1/20
99/99 [==============================] - 30s 275ms/step - loss: 1.6758 - accuracy: 0.2914 - val_loss: 1.2897 - val_accuracy: 0.5075
Epoch 2/20
99/99 [==============================] - 26s 268ms/step - loss: 1.2631 - accuracy: 0.5006 - val_loss: 1.2323 - val_accuracy: 0.4957
Epoch 3/20
99/99 [==============================] - 26s 267ms/step - loss: 1.1246 - accuracy: 0.5546 - val_loss: 0.9065 - val_accuracy: 0.6241
Epoch 4/20
99/99 [==============================] - 26s 267ms/step - loss: 1.0342 - accuracy: 0.5984 - val_loss: 0.9818 - val_accuracy: 0.6248
Epoch 5/20
99/99 [==============================] - 26s 268ms/step - loss: 0.9861 - accuracy: 0.6158 - val_loss: 0.7737 - val_accuracy: 0.7140
Epoch 6/20
99/99 [==============================] - 26s 267ms/step - loss: 0.9101 - accuracy: 0.6512 - val_loss: 0.9507 - val_accuracy: 0.6113
Epoch 7/20
99/99 [==============================] - 26s 267ms/step - loss: 0.8622 - accuracy: 0.6741 - val_loss: 0.9806 - val_accuracy: 0.6

In [175]:
time.sleep(400)

In [176]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

18
Epoch 1/20
104/104 [==============================] - 31s 272ms/step - loss: 1.6960 - accuracy: 0.2600 - val_loss: 1.1708 - val_accuracy: 0.5321
Epoch 2/20
104/104 [==============================] - 28s 268ms/step - loss: 1.2763 - accuracy: 0.4926 - val_loss: 1.1359 - val_accuracy: 0.5603
Epoch 3/20
104/104 [==============================] - 28s 269ms/step - loss: 1.1215 - accuracy: 0.5610 - val_loss: 0.9195 - val_accuracy: 0.6434
Epoch 4/20
104/104 [==============================] - 28s 268ms/step - loss: 1.0221 - accuracy: 0.6066 - val_loss: 0.8865 - val_accuracy: 0.6616
Epoch 5/20
104/104 [==============================] - 28s 268ms/step - loss: 0.9431 - accuracy: 0.6472 - val_loss: 1.0995 - val_accuracy: 0.5681
Epoch 6/20
104/104 [==============================] - 28s 268ms/step - loss: 0.8796 - accuracy: 0.6739 - val_loss: 0.9078 - val_accuracy: 0.6476
Epoch 7/20
104/104 [==============================] - 28s 270ms/step - loss: 0.8220 - accuracy: 0.6965 - val_loss: 0.7405 - val

In [177]:
time.sleep(400)

In [178]:
print(n)#

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

19
Epoch 1/20
110/110 [==============================] - 37s 302ms/step - loss: 1.6877 - accuracy: 0.2768 - val_loss: 1.1376 - val_accuracy: 0.5621
Epoch 2/20
110/110 [==============================] - 33s 298ms/step - loss: 1.3042 - accuracy: 0.4821 - val_loss: 1.1405 - val_accuracy: 0.5353
Epoch 3/20
110/110 [==============================] - 33s 299ms/step - loss: 1.1133 - accuracy: 0.5613 - val_loss: 0.9220 - val_accuracy: 0.6141
Epoch 4/20
110/110 [==============================] - 33s 299ms/step - loss: 1.0187 - accuracy: 0.6011 - val_loss: 0.9414 - val_accuracy: 0.6359
Epoch 5/20
110/110 [==============================] - 33s 298ms/step - loss: 0.9341 - accuracy: 0.6400 - val_loss: 0.8073 - val_accuracy: 0.7093
Epoch 6/20
110/110 [==============================] - 33s 298ms/step - loss: 0.8914 - accuracy: 0.6609 - val_loss: 0.9078 - val_accuracy: 0.6665
Epoch 7/20
110/110 [==============================] - 33s 298ms/step - loss: 0.8285 - accuracy: 0.6904 - val_loss: 0.7142 - val

In [179]:
time.sleep(400)

In [180]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

20
Epoch 1/20
112/112 [==============================] - 33s 271ms/step - loss: 1.6877 - accuracy: 0.2662 - val_loss: 1.2572 - val_accuracy: 0.5096
Epoch 2/20
112/112 [==============================] - 30s 266ms/step - loss: 1.2561 - accuracy: 0.5058 - val_loss: 1.4076 - val_accuracy: 0.4490
Epoch 3/20
112/112 [==============================] - 30s 268ms/step - loss: 1.1714 - accuracy: 0.5428 - val_loss: 1.1998 - val_accuracy: 0.5111
Epoch 4/20
112/112 [==============================] - 30s 266ms/step - loss: 1.0256 - accuracy: 0.6013 - val_loss: 1.1350 - val_accuracy: 0.5792
Epoch 5/20
112/112 [==============================] - 30s 267ms/step - loss: 0.9479 - accuracy: 0.6464 - val_loss: 1.3604 - val_accuracy: 0.5185
Epoch 6/20
112/112 [==============================] - 30s 266ms/step - loss: 0.8937 - accuracy: 0.6629 - val_loss: 1.1542 - val_accuracy: 0.5667
Epoch 7/20
112/112 [==============================] - 30s 268ms/step - loss: 0.8340 - accuracy: 0.6916 - val_loss: 0.9117 - val

In [181]:
new_model_random_dir  = "D:/models/aug_22/"+dataset+"/C1/"+dataset+"_model_c1_aug_random_e1"

i=0

for model in models_random:
    model.save(new_model_random_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_random_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_random_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_random_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_random_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_random_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_random_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_random_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_random_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/mo

In [ ]:
loading = False

models_random = []

if loading:
    for i in range(20):
        model_random_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_random_e1_"+str(i)
        print(model_random_dir)
        model =utils.My_model('gtsrb',True,model_random_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_random.append(model)
       

In [182]:
try:
    del random_indexes
    del top_images_by_random
    del top_labels_by_random
    del image_sets_random
    del label_sets_random
    del models_random
except:
    print("Error")

In [183]:
gc.collect()

105922

## Training guided by NC

In [184]:
# NC
nc_values = []
for i in range(1,6):
    #save_dir = "C:/Users/fjdur/Desktop/upc/project_notebooks/github_project/DL_notebooks/NC_values/nc_values_"+str(i)+".npy"
    save_dir = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/"+dataset+"_nc_values_"+str(i)+".npy"

    #print(save_dir_rand)
    tmp_values = np.load(save_dir)
    #print(tmp_values.shape)
    nc_values = np.append(nc_values,tmp_values)

In [185]:
nc_values.shape

(14224,)

In [186]:

top_images_by_nc = utils.get_x_of_indexes(list(np.flip(np.argsort(nc_values))),x_train_and_adversary)
top_labels_by_nc = utils.get_x_of_indexes(list(np.flip(np.argsort(nc_values))),y_train_and_adversary)


In [187]:
m = n_data_points
n = 0
image_sets_nc = []
label_sets_nc = []


for i in range((len(top_images_by_nc)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_nc)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_nc)%m))
        top_images_by_nc_n = np.array(top_images_by_nc[:n+m+(len(top_images_by_nc)%m)])
        top_labels_by_nc_n = np.array(top_labels_by_nc[:n+m+(len(top_images_by_nc)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_nc_n = np.array(top_images_by_nc[:n+m])
        top_labels_by_nc_n = np.array(top_labels_by_nc[:n+m])
    image_sets_nc.append(top_images_by_nc_n)
    label_sets_nc.append(top_labels_by_nc_n)
    print(len(top_images_by_nc_n))
    n += m




0 :
0  ->  700
700
1 :
0  ->  1400
1400
2 :
0  ->  2100
2100
3 :
0  ->  2800
2800
4 :
0  ->  3500
3500
5 :
0  ->  4200
4200
6 :
0  ->  4900
4900
7 :
0  ->  5600
5600
8 :
0  ->  6300
6300
9 :
0  ->  7000
7000
10 :
0  ->  7700
7700
11 :
0  ->  8400
8400
12 :
0  ->  9100
9100
13 :
0  ->  9800
9800
14 :
0  ->  10500
10500
15 :
0  ->  11200
11200
16 :
0  ->  11900
11900
17 :
0  ->  12600
12600
18 :
0  ->  13300
13300
19 :
0  ->  14000
14000
20 :
Last
0  ->  14924
14224


In [188]:
print(model_dir)

models_nc = []
for i in range(len(label_sets_nc)):
    print(i,":")
    model = utils.My_model(dataset,False,None)
    model.compile_model()
    models_nc.append(model)


C:/Users/fjdur/Documents/upc-july/models/intel_model_21_10
0 :
Model compiled
1 :
Model compiled
2 :
Model compiled
3 :
Model compiled
4 :
Model compiled
5 :
Model compiled
6 :
Model compiled
7 :
Model compiled
8 :
Model compiled
9 :
Model compiled
10 :
Model compiled
11 :
Model compiled
12 :
Model compiled
13 :
Model compiled
14 :
Model compiled
15 :
Model compiled
16 :
Model compiled
17 :
Model compiled
18 :
Model compiled
19 :
Model compiled
20 :
Model compiled


In [189]:
n=0
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

0
Epoch 1/20
6/6 [==============================] - 6s 636ms/step - loss: 2.0411 - accuracy: 0.3157 - val_loss: 2.0592 - val_accuracy: 0.1712
Epoch 2/20
6/6 [==============================] - 3s 531ms/step - loss: 1.2491 - accuracy: 0.5434 - val_loss: 1.9234 - val_accuracy: 0.1712
Epoch 3/20
6/6 [==============================] - 3s 522ms/step - loss: 1.2941 - accuracy: 0.5308 - val_loss: 2.4797 - val_accuracy: 0.1712
Epoch 4/20
6/6 [==============================] - 3s 529ms/step - loss: 1.2982 - accuracy: 0.5510 - val_loss: 1.7978 - val_accuracy: 0.2568
Epoch 5/20
6/6 [==============================] - 3s 538ms/step - loss: 1.3033 - accuracy: 0.5650 - val_loss: 2.1531 - val_accuracy: 0.2632
Epoch 6/20
6/6 [==============================] - 3s 532ms/step - loss: 1.1684 - accuracy: 0.5711 - val_loss: 2.0946 - val_accuracy: 0.2343
Epoch 7/20
6/6 [==============================] - 3s 583ms/step - loss: 1.0730 - accuracy: 0.6111 - val_loss: 2.4699 - val_accuracy: 0.2382
Epoch 8/20
6/6 [==

In [190]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

1
Epoch 1/20
11/11 [==============================] - 8s 467ms/step - loss: 1.6630 - accuracy: 0.3278 - val_loss: 2.0175 - val_accuracy: 0.1715
Epoch 2/20
11/11 [==============================] - 5s 479ms/step - loss: 1.4074 - accuracy: 0.4405 - val_loss: 2.4297 - val_accuracy: 0.2044
Epoch 3/20
11/11 [==============================] - 5s 492ms/step - loss: 1.3225 - accuracy: 0.4832 - val_loss: 1.7955 - val_accuracy: 0.2746
Epoch 4/20
11/11 [==============================] - 5s 475ms/step - loss: 1.2225 - accuracy: 0.5732 - val_loss: 1.7556 - val_accuracy: 0.3274
Epoch 5/20
11/11 [==============================] - 5s 465ms/step - loss: 1.2103 - accuracy: 0.5855 - val_loss: 1.8086 - val_accuracy: 0.3509
Epoch 6/20
11/11 [==============================] - 6s 540ms/step - loss: 1.1318 - accuracy: 0.5854 - val_loss: 2.2245 - val_accuracy: 0.2810
Epoch 7/20
11/11 [==============================] - 5s 472ms/step - loss: 1.0262 - accuracy: 0.6427 - val_loss: 2.0720 - val_accuracy: 0.3934
Epoc

In [191]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

2
Epoch 1/20
17/17 [==============================] - 10s 430ms/step - loss: 1.6178 - accuracy: 0.3319 - val_loss: 1.9266 - val_accuracy: 0.2293
Epoch 2/20
17/17 [==============================] - 7s 430ms/step - loss: 1.4067 - accuracy: 0.4915 - val_loss: 1.7019 - val_accuracy: 0.2668
Epoch 3/20
17/17 [==============================] - 7s 429ms/step - loss: 1.2443 - accuracy: 0.5317 - val_loss: 1.7060 - val_accuracy: 0.3484
Epoch 4/20
17/17 [==============================] - 8s 452ms/step - loss: 1.1775 - accuracy: 0.5631 - val_loss: 1.7653 - val_accuracy: 0.3798
Epoch 5/20
17/17 [==============================] - 7s 425ms/step - loss: 1.1795 - accuracy: 0.5850 - val_loss: 1.9184 - val_accuracy: 0.3088
Epoch 6/20
17/17 [==============================] - 7s 432ms/step - loss: 1.0834 - accuracy: 0.6056 - val_loss: 1.6302 - val_accuracy: 0.4123
Epoch 7/20
17/17 [==============================] - 7s 435ms/step - loss: 1.1065 - accuracy: 0.5972 - val_loss: 1.9438 - val_accuracy: 0.3581
Epo

In [192]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

3
Epoch 1/20
22/22 [==============================] - 13s 417ms/step - loss: 1.5916 - accuracy: 0.3238 - val_loss: 1.7629 - val_accuracy: 0.2347
Epoch 2/20
22/22 [==============================] - 9s 422ms/step - loss: 1.3238 - accuracy: 0.4900 - val_loss: 1.8803 - val_accuracy: 0.2721
Epoch 3/20
22/22 [==============================] - 9s 422ms/step - loss: 1.2187 - accuracy: 0.5177 - val_loss: 1.7301 - val_accuracy: 0.3199
Epoch 4/20
22/22 [==============================] - 9s 406ms/step - loss: 1.1491 - accuracy: 0.5542 - val_loss: 1.7159 - val_accuracy: 0.4030
Epoch 5/20
22/22 [==============================] - 9s 424ms/step - loss: 1.1008 - accuracy: 0.5898 - val_loss: 1.5451 - val_accuracy: 0.4526
Epoch 6/20
22/22 [==============================] - 9s 404ms/step - loss: 1.0761 - accuracy: 0.6061 - val_loss: 1.5772 - val_accuracy: 0.4440
Epoch 7/20
22/22 [==============================] - 9s 412ms/step - loss: 1.0199 - accuracy: 0.6476 - val_loss: 1.5194 - val_accuracy: 0.4404
Epo

In [193]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

4
Epoch 1/20
28/28 [==============================] - 14s 367ms/step - loss: 1.5934 - accuracy: 0.3259 - val_loss: 1.7407 - val_accuracy: 0.1847
Epoch 2/20
28/28 [==============================] - 11s 379ms/step - loss: 1.3422 - accuracy: 0.4426 - val_loss: 1.6263 - val_accuracy: 0.4108
Epoch 3/20
28/28 [==============================] - 10s 375ms/step - loss: 1.2397 - accuracy: 0.5031 - val_loss: 1.7243 - val_accuracy: 0.4148
Epoch 4/20
28/28 [==============================] - 10s 374ms/step - loss: 1.1823 - accuracy: 0.5461 - val_loss: 1.5106 - val_accuracy: 0.4604
Epoch 5/20
28/28 [==============================] - 10s 376ms/step - loss: 1.1013 - accuracy: 0.5764 - val_loss: 1.6614 - val_accuracy: 0.4287
Epoch 6/20
28/28 [==============================] - 10s 377ms/step - loss: 1.0477 - accuracy: 0.6131 - val_loss: 1.5577 - val_accuracy: 0.4508
Epoch 7/20
28/28 [==============================] - 10s 375ms/step - loss: 0.9939 - accuracy: 0.6281 - val_loss: 1.4192 - val_accuracy: 0.52

In [194]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

5
Epoch 1/20
33/33 [==============================] - 16s 372ms/step - loss: 1.6284 - accuracy: 0.3059 - val_loss: 1.8003 - val_accuracy: 0.2372
Epoch 2/20
33/33 [==============================] - 13s 391ms/step - loss: 1.3305 - accuracy: 0.4665 - val_loss: 1.8343 - val_accuracy: 0.3274
Epoch 3/20
33/33 [==============================] - 13s 408ms/step - loss: 1.2569 - accuracy: 0.5170 - val_loss: 2.2268 - val_accuracy: 0.2846
Epoch 4/20
33/33 [==============================] - 13s 397ms/step - loss: 1.1835 - accuracy: 0.5443 - val_loss: 1.4828 - val_accuracy: 0.5004
Epoch 5/20
33/33 [==============================] - 13s 396ms/step - loss: 1.1379 - accuracy: 0.5846 - val_loss: 1.4357 - val_accuracy: 0.4893
Epoch 6/20
33/33 [==============================] - 13s 382ms/step - loss: 1.0067 - accuracy: 0.6350 - val_loss: 1.4973 - val_accuracy: 0.5007
Epoch 7/20
33/33 [==============================] - 12s 380ms/step - loss: 0.9389 - accuracy: 0.6610 - val_loss: 1.7308 - val_accuracy: 0.46

In [195]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

6
Epoch 1/20
39/39 [==============================] - 18s 367ms/step - loss: 1.5687 - accuracy: 0.3057 - val_loss: 1.7513 - val_accuracy: 0.2703
Epoch 2/20
39/39 [==============================] - 15s 376ms/step - loss: 1.3641 - accuracy: 0.4388 - val_loss: 1.9620 - val_accuracy: 0.3680
Epoch 3/20
39/39 [==============================] - 14s 369ms/step - loss: 1.2745 - accuracy: 0.4819 - val_loss: 2.0310 - val_accuracy: 0.3277
Epoch 4/20
39/39 [==============================] - 14s 368ms/step - loss: 1.1706 - accuracy: 0.5350 - val_loss: 1.5771 - val_accuracy: 0.4822
Epoch 5/20
39/39 [==============================] - 14s 367ms/step - loss: 1.0663 - accuracy: 0.6013 - val_loss: 1.4463 - val_accuracy: 0.4408
Epoch 6/20
39/39 [==============================] - 14s 367ms/step - loss: 0.9826 - accuracy: 0.6467 - val_loss: 1.3856 - val_accuracy: 0.5275
Epoch 7/20
39/39 [==============================] - 14s 368ms/step - loss: 0.9345 - accuracy: 0.6691 - val_loss: 1.5645 - val_accuracy: 0.50

In [196]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

7
Epoch 1/20
44/44 [==============================] - 20s 367ms/step - loss: 1.5885 - accuracy: 0.3103 - val_loss: 1.6453 - val_accuracy: 0.3941
Epoch 2/20
44/44 [==============================] - 16s 367ms/step - loss: 1.3277 - accuracy: 0.4753 - val_loss: 1.6936 - val_accuracy: 0.3923
Epoch 3/20
44/44 [==============================] - 16s 366ms/step - loss: 1.2511 - accuracy: 0.5074 - val_loss: 1.6312 - val_accuracy: 0.4165
Epoch 4/20
44/44 [==============================] - 16s 365ms/step - loss: 1.1730 - accuracy: 0.5525 - val_loss: 1.4106 - val_accuracy: 0.5100
Epoch 5/20
44/44 [==============================] - 16s 369ms/step - loss: 1.0746 - accuracy: 0.5946 - val_loss: 1.3334 - val_accuracy: 0.4964
Epoch 6/20
44/44 [==============================] - 16s 364ms/step - loss: 0.9996 - accuracy: 0.6377 - val_loss: 1.3336 - val_accuracy: 0.5467
Epoch 7/20
44/44 [==============================] - 16s 365ms/step - loss: 0.9066 - accuracy: 0.6779 - val_loss: 1.4077 - val_accuracy: 0.55

In [197]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

8
Epoch 1/20
50/50 [==============================] - 23s 373ms/step - loss: 1.6526 - accuracy: 0.3017 - val_loss: 1.6631 - val_accuracy: 0.3805
Epoch 2/20
50/50 [==============================] - 18s 367ms/step - loss: 1.3586 - accuracy: 0.4448 - val_loss: 1.6229 - val_accuracy: 0.4272
Epoch 3/20
50/50 [==============================] - 19s 376ms/step - loss: 1.2972 - accuracy: 0.4800 - val_loss: 1.4283 - val_accuracy: 0.4572
Epoch 4/20
50/50 [==============================] - 18s 363ms/step - loss: 1.1727 - accuracy: 0.5470 - val_loss: 1.2904 - val_accuracy: 0.5178
Epoch 5/20
50/50 [==============================] - 18s 370ms/step - loss: 1.0726 - accuracy: 0.6039 - val_loss: 1.3166 - val_accuracy: 0.5182
Epoch 6/20
50/50 [==============================] - 18s 364ms/step - loss: 1.0266 - accuracy: 0.6235 - val_loss: 1.3433 - val_accuracy: 0.5321
Epoch 7/20
50/50 [==============================] - 18s 361ms/step - loss: 0.9679 - accuracy: 0.6435 - val_loss: 1.1514 - val_accuracy: 0.58

In [198]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

9
Epoch 1/20
55/55 [==============================] - 25s 381ms/step - loss: 1.6703 - accuracy: 0.2926 - val_loss: 1.6871 - val_accuracy: 0.3720
Epoch 2/20
55/55 [==============================] - 21s 376ms/step - loss: 1.3564 - accuracy: 0.4604 - val_loss: 1.3931 - val_accuracy: 0.4608
Epoch 3/20
55/55 [==============================] - 21s 373ms/step - loss: 1.2478 - accuracy: 0.5191 - val_loss: 1.4481 - val_accuracy: 0.4747
Epoch 4/20
55/55 [==============================] - 21s 375ms/step - loss: 1.1631 - accuracy: 0.5571 - val_loss: 1.3324 - val_accuracy: 0.4679
Epoch 5/20
55/55 [==============================] - 20s 366ms/step - loss: 1.0467 - accuracy: 0.6081 - val_loss: 1.1260 - val_accuracy: 0.5867
Epoch 6/20
55/55 [==============================] - 20s 363ms/step - loss: 1.0078 - accuracy: 0.6440 - val_loss: 1.1213 - val_accuracy: 0.5767
Epoch 7/20
55/55 [==============================] - 20s 367ms/step - loss: 0.9029 - accuracy: 0.6644 - val_loss: 1.7200 - val_accuracy: 0.43

In [199]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

10
Epoch 1/20
61/61 [==============================] - 26s 358ms/step - loss: 1.6204 - accuracy: 0.3068 - val_loss: 1.7855 - val_accuracy: 0.3552
Epoch 2/20
61/61 [==============================] - 22s 366ms/step - loss: 1.3558 - accuracy: 0.4539 - val_loss: 1.5306 - val_accuracy: 0.4176
Epoch 3/20
61/61 [==============================] - 22s 365ms/step - loss: 1.2600 - accuracy: 0.4968 - val_loss: 2.0357 - val_accuracy: 0.2889
Epoch 4/20
61/61 [==============================] - 22s 367ms/step - loss: 1.1723 - accuracy: 0.5470 - val_loss: 1.5922 - val_accuracy: 0.4954
Epoch 5/20
61/61 [==============================] - 22s 364ms/step - loss: 1.0603 - accuracy: 0.5960 - val_loss: 1.2322 - val_accuracy: 0.5456
Epoch 6/20
61/61 [==============================] - 22s 361ms/step - loss: 1.0026 - accuracy: 0.6258 - val_loss: 1.4013 - val_accuracy: 0.5749
Epoch 7/20
61/61 [==============================] - 22s 367ms/step - loss: 0.9438 - accuracy: 0.6507 - val_loss: 1.0582 - val_accuracy: 0.6

In [200]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

11
Epoch 1/20
66/66 [==============================] - 29s 373ms/step - loss: 1.6272 - accuracy: 0.2848 - val_loss: 1.6193 - val_accuracy: 0.3730
Epoch 2/20
66/66 [==============================] - 24s 368ms/step - loss: 1.3890 - accuracy: 0.4432 - val_loss: 1.3903 - val_accuracy: 0.4305
Epoch 3/20
66/66 [==============================] - 24s 362ms/step - loss: 1.2595 - accuracy: 0.5057 - val_loss: 1.2003 - val_accuracy: 0.5300
Epoch 4/20
66/66 [==============================] - 24s 368ms/step - loss: 1.1222 - accuracy: 0.5682 - val_loss: 1.2043 - val_accuracy: 0.5221
Epoch 5/20
66/66 [==============================] - 24s 360ms/step - loss: 1.0400 - accuracy: 0.6080 - val_loss: 1.0675 - val_accuracy: 0.6248
Epoch 6/20
66/66 [==============================] - 24s 359ms/step - loss: 0.9381 - accuracy: 0.6584 - val_loss: 0.9459 - val_accuracy: 0.6534
Epoch 7/20
66/66 [==============================] - 24s 358ms/step - loss: 0.9118 - accuracy: 0.6695 - val_loss: 0.9333 - val_accuracy: 0.6

In [201]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

12
Epoch 1/20
72/72 [==============================] - 29s 352ms/step - loss: 1.6704 - accuracy: 0.2822 - val_loss: 2.1293 - val_accuracy: 0.2068
Epoch 2/20
72/72 [==============================] - 25s 349ms/step - loss: 1.4257 - accuracy: 0.4288 - val_loss: 1.3878 - val_accuracy: 0.4650
Epoch 3/20
72/72 [==============================] - 26s 357ms/step - loss: 1.2190 - accuracy: 0.5089 - val_loss: 1.8429 - val_accuracy: 0.4005
Epoch 4/20
72/72 [==============================] - 25s 350ms/step - loss: 1.1392 - accuracy: 0.5724 - val_loss: 1.2969 - val_accuracy: 0.5096
Epoch 5/20
72/72 [==============================] - 25s 350ms/step - loss: 1.0214 - accuracy: 0.6205 - val_loss: 1.2398 - val_accuracy: 0.4996
Epoch 6/20
72/72 [==============================] - 25s 350ms/step - loss: 0.9512 - accuracy: 0.6412 - val_loss: 1.1409 - val_accuracy: 0.5752
Epoch 7/20
72/72 [==============================] - 26s 356ms/step - loss: 0.9138 - accuracy: 0.6717 - val_loss: 1.3951 - val_accuracy: 0.6

In [202]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

13
Epoch 1/20
77/77 [==============================] - 30s 340ms/step - loss: 1.6565 - accuracy: 0.2895 - val_loss: 1.6246 - val_accuracy: 0.4001
Epoch 2/20
77/77 [==============================] - 26s 343ms/step - loss: 1.3364 - accuracy: 0.4763 - val_loss: 1.2218 - val_accuracy: 0.5025
Epoch 3/20
77/77 [==============================] - 26s 341ms/step - loss: 1.1678 - accuracy: 0.5387 - val_loss: 1.0141 - val_accuracy: 0.6134
Epoch 4/20
77/77 [==============================] - 27s 352ms/step - loss: 1.1016 - accuracy: 0.5709 - val_loss: 0.9479 - val_accuracy: 0.6476
Epoch 5/20
77/77 [==============================] - 26s 344ms/step - loss: 1.0039 - accuracy: 0.6233 - val_loss: 0.9975 - val_accuracy: 0.6255
Epoch 6/20
77/77 [==============================] - 27s 346ms/step - loss: 0.9293 - accuracy: 0.6592 - val_loss: 0.9702 - val_accuracy: 0.6644
Epoch 7/20
77/77 [==============================] - 27s 352ms/step - loss: 0.8531 - accuracy: 0.6857 - val_loss: 0.8134 - val_accuracy: 0.7

In [203]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

14
Epoch 1/20
83/83 [==============================] - 32s 338ms/step - loss: 1.6383 - accuracy: 0.3222 - val_loss: 1.3866 - val_accuracy: 0.4490
Epoch 2/20
83/83 [==============================] - 28s 340ms/step - loss: 1.3214 - accuracy: 0.4684 - val_loss: 2.0187 - val_accuracy: 0.3192
Epoch 3/20
83/83 [==============================] - 28s 342ms/step - loss: 1.1746 - accuracy: 0.5455 - val_loss: 1.1007 - val_accuracy: 0.5581
Epoch 4/20
83/83 [==============================] - 29s 346ms/step - loss: 1.0388 - accuracy: 0.5982 - val_loss: 1.4189 - val_accuracy: 0.4936
Epoch 5/20
83/83 [==============================] - 29s 352ms/step - loss: 0.9964 - accuracy: 0.6326 - val_loss: 2.2766 - val_accuracy: 0.3759
Epoch 6/20
83/83 [==============================] - 28s 338ms/step - loss: 0.9892 - accuracy: 0.6475 - val_loss: 1.2537 - val_accuracy: 0.5011
Epoch 7/20
83/83 [==============================] - 28s 338ms/step - loss: 0.8564 - accuracy: 0.6875 - val_loss: 1.0163 - val_accuracy: 0.6

In [204]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

15
Epoch 1/20
88/88 [==============================] - 34s 338ms/step - loss: 1.6703 - accuracy: 0.2892 - val_loss: 1.3077 - val_accuracy: 0.4747
Epoch 2/20
88/88 [==============================] - 30s 340ms/step - loss: 1.2923 - accuracy: 0.4955 - val_loss: 1.2134 - val_accuracy: 0.5143
Epoch 3/20
88/88 [==============================] - 30s 341ms/step - loss: 1.1475 - accuracy: 0.5457 - val_loss: 1.0802 - val_accuracy: 0.5913
Epoch 4/20
88/88 [==============================] - 30s 340ms/step - loss: 1.0475 - accuracy: 0.5965 - val_loss: 1.2267 - val_accuracy: 0.5760
Epoch 5/20
88/88 [==============================] - 30s 344ms/step - loss: 0.9454 - accuracy: 0.6394 - val_loss: 0.9774 - val_accuracy: 0.6437
Epoch 6/20
88/88 [==============================] - 30s 340ms/step - loss: 0.8784 - accuracy: 0.6723 - val_loss: 1.0664 - val_accuracy: 0.6170
Epoch 7/20
88/88 [==============================] - 30s 341ms/step - loss: 0.8457 - accuracy: 0.6894 - val_loss: 0.9445 - val_accuracy: 0.6

In [205]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

16
Epoch 1/20
93/93 [==============================] - 36s 340ms/step - loss: 1.6765 - accuracy: 0.2911 - val_loss: 1.2918 - val_accuracy: 0.4936
Epoch 2/20
93/93 [==============================] - 32s 341ms/step - loss: 1.3114 - accuracy: 0.4926 - val_loss: 1.0971 - val_accuracy: 0.5788
Epoch 3/20
93/93 [==============================] - 32s 340ms/step - loss: 1.1231 - accuracy: 0.5656 - val_loss: 0.9549 - val_accuracy: 0.6409
Epoch 4/20
93/93 [==============================] - 32s 340ms/step - loss: 1.0231 - accuracy: 0.6086 - val_loss: 1.0818 - val_accuracy: 0.5942
Epoch 5/20
93/93 [==============================] - 33s 352ms/step - loss: 0.9236 - accuracy: 0.6528 - val_loss: 1.1213 - val_accuracy: 0.5720
Epoch 6/20
93/93 [==============================] - 32s 341ms/step - loss: 0.8791 - accuracy: 0.6714 - val_loss: 0.8600 - val_accuracy: 0.6797
Epoch 7/20
93/93 [==============================] - 32s 341ms/step - loss: 0.8300 - accuracy: 0.6971 - val_loss: 0.8401 - val_accuracy: 0.6

In [206]:
time.sleep(400)

In [207]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

17
Epoch 1/20
99/99 [==============================] - 49s 422ms/step - loss: 1.6924 - accuracy: 0.2827 - val_loss: 1.8640 - val_accuracy: 0.3566
Epoch 2/20
99/99 [==============================] - 42s 425ms/step - loss: 1.3042 - accuracy: 0.4798 - val_loss: 1.0237 - val_accuracy: 0.6056
Epoch 3/20
99/99 [==============================] - 43s 438ms/step - loss: 1.1113 - accuracy: 0.5661 - val_loss: 1.0801 - val_accuracy: 0.6020
Epoch 4/20
99/99 [==============================] - 43s 438ms/step - loss: 0.9985 - accuracy: 0.6208 - val_loss: 0.8828 - val_accuracy: 0.6690
Epoch 5/20
99/99 [==============================] - 42s 420ms/step - loss: 0.9163 - accuracy: 0.6538 - val_loss: 0.9841 - val_accuracy: 0.6733
Epoch 6/20
99/99 [==============================] - 43s 431ms/step - loss: 0.8493 - accuracy: 0.6824 - val_loss: 0.7368 - val_accuracy: 0.7315
Epoch 7/20
99/99 [==============================] - 44s 441ms/step - loss: 0.8056 - accuracy: 0.6973 - val_loss: 0.8655 - val_accuracy: 0.6

In [208]:
time.sleep(400)

In [209]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

18
Epoch 1/20
104/104 [==============================] - 54s 453ms/step - loss: 1.6640 - accuracy: 0.2980 - val_loss: 1.3158 - val_accuracy: 0.4850
Epoch 2/20
104/104 [==============================] - 46s 440ms/step - loss: 1.2330 - accuracy: 0.5136 - val_loss: 1.2683 - val_accuracy: 0.4989
Epoch 3/20
104/104 [==============================] - 46s 441ms/step - loss: 1.1021 - accuracy: 0.5711 - val_loss: 0.8813 - val_accuracy: 0.6469
Epoch 4/20
104/104 [==============================] - 46s 439ms/step - loss: 0.9748 - accuracy: 0.6223 - val_loss: 0.8160 - val_accuracy: 0.6933
Epoch 5/20
104/104 [==============================] - 45s 434ms/step - loss: 0.8909 - accuracy: 0.6636 - val_loss: 1.2125 - val_accuracy: 0.5745
Epoch 6/20
104/104 [==============================] - 45s 436ms/step - loss: 0.8404 - accuracy: 0.6866 - val_loss: 0.7123 - val_accuracy: 0.7468
Epoch 7/20
104/104 [==============================] - 54s 523ms/step - loss: 0.7887 - accuracy: 0.7114 - val_loss: 0.7973 - val

In [210]:
time.sleep(400)

In [211]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

19
Epoch 1/20
110/110 [==============================] - 101s 759ms/step - loss: 1.6720 - accuracy: 0.3014 - val_loss: 1.1409 - val_accuracy: 0.5578
Epoch 2/20
110/110 [==============================] - 77s 697ms/step - loss: 1.2439 - accuracy: 0.5069 - val_loss: 1.1687 - val_accuracy: 0.5342
Epoch 3/20
110/110 [==============================] - 78s 710ms/step - loss: 1.1203 - accuracy: 0.5631 - val_loss: 0.9267 - val_accuracy: 0.6241
Epoch 4/20
110/110 [==============================] - 79s 722ms/step - loss: 1.0010 - accuracy: 0.6151 - val_loss: 0.9322 - val_accuracy: 0.6230
Epoch 5/20
110/110 [==============================] - 81s 740ms/step - loss: 0.9407 - accuracy: 0.6410 - val_loss: 1.1682 - val_accuracy: 0.5717
Epoch 6/20
110/110 [==============================] - 62s 560ms/step - loss: 0.8919 - accuracy: 0.6688 - val_loss: 0.7746 - val_accuracy: 0.7175
Epoch 7/20
110/110 [==============================] - 77s 705ms/step - loss: 0.8291 - accuracy: 0.6926 - val_loss: 0.7368 - va

In [212]:
time.sleep(400)

In [213]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 128)
n = n+1

20
Epoch 1/20
112/112 [==============================] - 58s 456ms/step - loss: 1.6916 - accuracy: 0.2578 - val_loss: 1.2195 - val_accuracy: 0.5542
Epoch 2/20
112/112 [==============================] - 51s 459ms/step - loss: 1.2473 - accuracy: 0.5069 - val_loss: 1.0178 - val_accuracy: 0.5874
Epoch 3/20
112/112 [==============================] - 52s 465ms/step - loss: 1.0996 - accuracy: 0.5691 - val_loss: 1.0646 - val_accuracy: 0.5321
Epoch 4/20
112/112 [==============================] - 50s 446ms/step - loss: 1.0035 - accuracy: 0.6145 - val_loss: 0.9082 - val_accuracy: 0.6555
Epoch 5/20
112/112 [==============================] - 50s 445ms/step - loss: 0.9090 - accuracy: 0.6570 - val_loss: 0.8310 - val_accuracy: 0.6787
Epoch 6/20
112/112 [==============================] - 50s 445ms/step - loss: 0.8729 - accuracy: 0.6749 - val_loss: 0.9271 - val_accuracy: 0.6591
Epoch 7/20
112/112 [==============================] - 49s 435ms/step - loss: 0.8130 - accuracy: 0.7021 - val_loss: 1.0922 - val

In [214]:
new_model_nc_dir  = "D:/models/aug_22/"+dataset+"/C1/"+dataset+"_model_c1_aug_nc_e1"

i=0

for model in models_nc:
    model.save(new_model_nc_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_nc_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_nc_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_nc_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_nc_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_nc_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_nc_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_nc_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model_c1_aug_nc_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/intel/C1/intel_model

In [ ]:
loading = False

models_nc = []

if loading:
    for i in range(20):
        model_nc_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_nc_e1"+str(i)
        print(model_nc_dir)
        model =utils.My_model(dataset,True,model_nc_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_nc.append(model)
       

In [ ]:
try:
    del nc_values
    del top_images_by_nc
    del top_labels_by_nc
    del image_sets_nc
    del label_sets_nc
    del models_nc
except:
    print("Error")

In [ ]:
gc.collect()

## Evaluating

In [ ]:
model_original.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [ ]:
# Metrics using adversarial test
evaluate_lsa_0 = []
evaluate_dsa_0 = []
evaluate_nc_0 = []
evaluate_random_0 = []

evaluate_lsa_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_dsa_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_nc_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_random_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))

for model in models_lsa:
    evaluate_lsa_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_dsa:
    evaluate_dsa_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_random:
    evaluate_random_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_nc:
    evaluate_nc_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))

In [ ]:
evaluate_lsa_0[0]=[0,0,0,0]
evaluate_dsa_0[0]=[0,0,0,0]
evaluate_random_0[0]=[0,0,0,0]
evaluate_nc_0[0]=[0,0,0,0]

In [ ]:
print(len(evaluate_lsa_0))
print(len(evaluate_dsa_0))
print(len(evaluate_random_0))
print(len(evaluate_nc_0))

In [ ]:
# Metrics using original test
evaluate_lsa_1 = []
evaluate_dsa_1 = []
evaluate_nc_1 = []
evaluate_random_1 = []

evaluate_lsa_1.append(model_original.evaluate(x_test,y_test))
evaluate_dsa_1.append(model_original.evaluate(x_test,y_test))
evaluate_nc_1.append(model_original.evaluate(x_test,y_test))
evaluate_random_1.append(model_original.evaluate(x_test,y_test))
print("lsa---")
for model in models_lsa:
    evaluate_lsa_1.append(model.evaluate(x_test,y_test))

print("dsa---")
    
for model in models_dsa:
    evaluate_dsa_1.append(model.evaluate(x_test,y_test))
    

print("random---")

for model in models_random:
    evaluate_random_1.append(model.evaluate(x_test,y_test))
    
print("nc---")

for model in models_nc:
    evaluate_nc_1.append(model.evaluate(x_test,y_test))


In [ ]:
evaluate_lsa_1[0]=[0,0,0,0]
evaluate_dsa_1[0]=[0,0,0,0]
evaluate_random_1[0]=[0,0,0,0]
evaluate_nc_1[0]=[0,0,0,0]

In [ ]:

print(len(evaluate_lsa_1))
print(len(evaluate_dsa_1))
print(len(evaluate_random_1))
print(len(evaluate_nc_1))

In [ ]:
type(evaluate_dsa_0[0][0])

In [ ]:
import pandas as pd

df_evaluate_lsa_0 = pd.DataFrame(np.array(evaluate_lsa_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_lsa_1 = pd.DataFrame(np.array(evaluate_lsa_1),columns=["loss","accuracy","precision","recall"])
df_evaluate_dsa_0 = pd.DataFrame(np.array(evaluate_dsa_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_dsa_1 = pd.DataFrame(np.array(evaluate_dsa_1),columns=["loss","accuracy","precision","recall"])

df_evaluate_random_0 = pd.DataFrame(np.array(evaluate_random_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_random_1 = pd.DataFrame(np.array(evaluate_random_1),columns=["loss","accuracy","precision","recall"])
df_evaluate_nc_0 = pd.DataFrame(np.array(evaluate_nc_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_nc_1 = pd.DataFrame(np.array(evaluate_nc_1),columns=["loss","accuracy","precision","recall"])


In [ ]:
# Original test set + adversarial set

metric ="accuracy"
accuracy_lsa_3 = (np.array(df_evaluate_lsa_0[metric])+np.array(df_evaluate_lsa_1[metric]))/2
accuracy_dsa_3 = (np.array(df_evaluate_dsa_0[metric])+np.array(df_evaluate_dsa_1[metric]))/2
accuracy_nc_3 = (np.array(df_evaluate_nc_0[metric])+np.array(df_evaluate_nc_1[metric]))/2
accuracy_random_3 = (np.array(df_evaluate_random_0[metric])+np.array(df_evaluate_random_1[metric]))/2


## Charts

In [ ]:
n_inputs = [1800*i for i in range(20)]
n_inputs.append(len(x_train_and_adversary))

In [ ]:
print(n_inputs)

In [ ]:
cd "C:/Users/fjdur/Desktop/gtsrb_graphs/e2_graphs_dec/"

In [ ]:
linestyles = ['solid','dotted','dashed','dashdot']
colors =['k','k','k','k']

In [ ]:
#configuration 1
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

#metric = "accuracy" # accuracy loss
my_metrics =["accuracy","loss","precision","recall"]

for metric in my_metrics:
    plt.clf()
    plt.plot(n_inputs,df_evaluate_lsa_0[metric],colors[0],linestyle=linestyles[0])
    plt.plot(n_inputs,df_evaluate_dsa_0[metric],colors[1],linestyle=linestyles[1])
    plt.plot(n_inputs,df_evaluate_random_0[metric],colors[2],linestyle=linestyles[2])
    plt.plot(n_inputs,df_evaluate_nc_0[metric],colors[3],linestyle=linestyles[3])


    legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



    plt.legend(handles=legend_elements)#
    plt.title(metric + " with test set of adversarial examples FGSM")
    plt.xlim([0, 37000])
    #plt.ylim([0, 1])
    plt.xlabel('number of inputs')
    plt.ylabel(metric)
    plt.savefig("gtsrb_c1_"+metric + "_0.svg")
    plt.show()

In [ ]:
#configuration 1
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

my_metrics =["accuracy","loss","precision","recall"]

for metric in my_metrics:
    
#metric = "accuracy" # accuracy loss
    plt.clf()
    plt.plot(n_inputs,df_evaluate_lsa_1[metric],colors[0],linestyle=linestyles[0])
    plt.plot(n_inputs,df_evaluate_dsa_1[metric],colors[1],linestyle=linestyles[1])
    plt.plot(n_inputs,df_evaluate_random_1[metric],colors[2],linestyle=linestyles[2])
    plt.plot(n_inputs,df_evaluate_nc_1[metric],colors[3],linestyle=linestyles[3])

    legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



    plt.legend(handles=legend_elements)#
    plt.title(metric + " with original set")
    plt.xlim([0, 37000])
    #plt.ylim([0, 1])
    plt.xlabel('number of inputs')
    plt.ylabel(metric)
    plt.savefig("gtsrb_c1_"+metric + "_1.svg")
    plt.show()

In [ ]:
#adversarial jsma test set
#configuration 3
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

plt.plot(n_inputs,accuracy_lsa_3,colors[0],linestyle=linestyles[0])
plt.plot(n_inputs,accuracy_dsa_3,colors[1],linestyle=linestyles[1])
plt.plot(n_inputs,accuracy_random_3,colors[2],linestyle=linestyles[2])
plt.plot(n_inputs,accuracy_nc_3,colors[3],linestyle=linestyles[3])


legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



plt.legend(handles=legend_elements)#
plt.title("accuracy with both sets")

print(n_inputs[np.argmax(accuracy_lsa_3)],accuracy_lsa_3.max())
print(n_inputs[np.argmax(accuracy_dsa_3)],accuracy_dsa_3.max())
print(n_inputs[np.argmax(accuracy_random_3)],accuracy_random_3.max())
print(n_inputs[np.argmax(accuracy_nc_3)],accuracy_nc_3.max())

plt.plot(n_inputs[np.argmax(accuracy_lsa_3)],accuracy_lsa_3.max(),'-kD')
plt.plot(n_inputs[np.argmax(accuracy_dsa_3)],accuracy_dsa_3.max(),'-ko')
plt.plot(n_inputs[np.argmax(accuracy_random_3)],accuracy_random_3.max(),'-kv')
plt.plot(n_inputs[np.argmax(accuracy_nc_3)],accuracy_nc_3.max(),'-kp')

plt.xlabel('number of inputs')
plt.ylabel('accuracy')
plt.xlim([0, 37000])
#plt.ylim([0, 1])

plt.savefig("gtsrb_c1_"+"accuracy" + "_both.svg")

plt.show()

## Saving models

In [ ]:
new_model_lsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_lsa_e2"
i=0

for model in models_lsa:
    model.save(new_model_lsa_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_dsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_dsa_e2"

i=0
for model in models_dsa:
    model.save(new_model_dsa_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_random_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_random_e2"

i=0
for model in models_random:
    model.save(new_model_random_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_nc_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_nc_e2"

i=0
for model in models_nc:
    model.save(new_model_nc_dir+"_"+str(i))
    i+=1